In [ ]:
#Codigo de Elias Coradini, Uruguay Primaria 2023

In [ ]:
library(readxl)
library(dplyr)
library("writexl")
library('stringr')
Uruguay <- read.csv("C:/Users/Elias-PC/Facultad/URUGUAY.csv")
Primaria<- read.csv("C:/Users/Elias-PC/Facultad/Primaria 2023 - matrícula primaria.xls.csv")
Primaria_por_dep <- read.csv("C:/Users/Elias-PC/Facultad/primaria 2023 por departamentos - xls_export (10).csv", encoding = "UTF-8")

In [ ]:
#Renombramos variables
Uruguay <- Uruguay %>%
rename(
    Departamento = departamento,
    Localidad = localidad,
    Turno = turno,
    Curso = curso,
    PeriodoEvaluatorio = periodoEvaluatorio,
    IdAlumno = idalumno,
    IdDivision = idDivision,
    División = ddivision,
    Multianio = esMultianio,
    ModEnsenianza = ModEnsenaza
  )

names(Uruguay)

In [ ]:
#Limpieza
dim(Uruguay)
Uruguay1 <- select(Uruguay, -observaciones)
dim(Uruguay1)

Uruguay_na<-na.omit(Uruguay1)
dim(Uruguay_na)

Uruguay_na <- subset(Uruguay_na, !(nota == "NULL" | nota == "PA"))
#NO BORRAR EPACS, PD Y USAR FILTER PARA SACAR PROMEDIOS. YA QUE ESTOS NO SON NUMEROS.

print(unique(Uruguay_na$PeriodoEvaluatorio))


In [ ]:
Uruguay_primaria <- filter(Uruguay_na, 
                           NivelEnsenanza == "PRIMARIO" &
                           anioLectivo == "2023" & 
                           (ModEnsenianza == "Comun" | ModEnsenianza == "Escuelas NINA") &
                           Multianio == "N", Modalidad == "Primaria")

Uruguay_primaria$nota <- gsub("PD", "EPAC", Uruguay_primaria$nota)
print(unique(Uruguay_primaria$nota))

ftable(Uruguay_primaria$Modalidad)

In [ ]:
#Cantidad de Escuelas
length(unique(Uruguay_primaria$CUE))

#Cantidad de escuelas cargadas
CUES_uruguay <- unique(Uruguay_primaria$CUE)

filtrado <- filter(Primaria, CUE %in% CUES_uruguay)
nrow(filtrado)

In [ ]:
#Matricula Total
Matriculas_uruguay <- filter(Primaria_por_dep, Departamento=="URUGUAY")

Matriculas_uruguay_FIX <- Matriculas_uruguay %>%
rename(
    Primero = X1er.Año.Grado,
    Segundo = X2do.Año.Grado,
    Tercero = X3er.Año.Grado,
    Cuarto = X4to.Año.Grado,
    Quinto = X5to.Año.Grado,
    Sexto = X6to.Año.Grado
)

#Limpieza
columnas <- c("Todos","Primero", "Segundo", "Tercero", "Cuarto", "Quinto", "Sexto")
for (x in columnas) {
    Matriculas_uruguay_FIX[[x]] <- gsub(",", "", Matriculas_uruguay_FIX[[x]]) #Les saco la , si es que tienen
    Matriculas_uruguay_FIX[[x]] <- as.integer(Matriculas_uruguay_FIX[[x]])} #Los paso a INT

head(Matriculas_uruguay_FIX)
#Escribimos el archivo corregido
path <- "C:/Users/Elias-PC/Facultad/primaria_pordep_fix.csv"
write.csv(Matriculas_uruguay_FIX, path)

# Sumar todos los de primero
for (x in columnas) {
    total_primero <- sum(Matriculas_uruguay_FIX[[x]])
    print(paste(x,total_primero))}

In [ ]:
PrimerT<-filter(Uruguay_primaria, PeriodoEvaluatorio=="Primer Trimestre")
dim(PrimerT)
PrimerT<-rename(PrimerT,NotasPrimerT=nota)

print(unique(PrimerT$ModEnsenianza))

SegundoT<-filter(Uruguay_primaria, PeriodoEvaluatorio== "Segundo Trimestre")
SegundoT<-rename(SegundoT, NotasSegundoT= nota)
dim(SegundoT)

print(unique(SegundoT$ModEnsenianza))

TercerT <- filter(Uruguay_primaria, PeriodoEvaluatorio == "Tercer Trimestre")
TercerT <- rename(TercerT, NotasTercerT = nota)
dim(TercerT)

unique(TercerT$ModEnsenianza)

prueba <- merge(x = PrimerT, y = SegundoT, by = c("Documento", "asignatura"))
dim(prueba)

unique(prueba$ModEnsenianza)

prueba1 <- merge(x = prueba, y = TercerT, by = c("Documento", "asignatura"))
dim(prueba1)

unique(prueba1$ModEnsenianza)

In [ ]:
notas_uruguay <- select(prueba1, 'CUE', 'Departamento', 'EsPrivada', 'Turno' , 'Curso', 'Documento', 'asignatura','IdAlumno', 'IdDivision','NivelEnsenanza', 'Multianio', 'Modalidad', 'ModEnsenianza','NotasPrimerT','NotasSegundoT', 'NotasTercerT')
dim(notas_uruguay)
tail(notas_uruguay,4)

In [ ]:
notas_uruguay$Promedio<-round((as.numeric(notas_uruguay$NotasPrimerT)+as.numeric(notas_uruguay$NotasSegundoT)+as.numeric(notas_uruguay$NotasTercerT))/3,2)

notas_uruguay_FIX <- unique(notas_uruguay) # FILTRO LOS REPETIDOS

notas_uruguay_FIX <- notas_uruguay_FIX %>%
    mutate(Resultado = case_when(
    NotasTercerT == "EPAC" ~ "En proceso", #Si en el tercer trimestre tiene EPAC, esta "En proceso"
    NotasTercerT >= 6 ~ "Aprobado", #Si tiene mas de 6 en el tercer trimestre esta Aprobado
    NotasTercerT == "10" ~ "Aprobado",
    Promedio >= 6 ~ "Aprobado",
    Promedio < 6  ~ "Rinde"))

#print(notas_uruguay_FIX[, c("NotasPrimerT","NotasSegundoT","NotasTercerT","Promedio","Resultado")])

write_xlsx(notas_uruguay_FIX, "C:/Users/Elias-PC/Facultad/Unotas_uruguay_FIX678945a.xlsx")
head(notas_uruguay,4)
head(notas_uruguay_FIX,4)

In [ ]:
notas_uruguay_FIX$Curso<-as.character(notas_uruguay_FIX$Curso)
notas_uruguay_FIX$Año_cursado<-str_sub(notas_uruguay_FIX$Curso,1, 1)
print(head(notas_uruguay_FIX$Año_cursado),2)
table(notas_uruguay_FIX$Año_cursado)
notas_uruguay_FIX$Año_cursado <- as.integer(notas_uruguay_FIX$Año_cursado)

unique(notas_uruguay_FIX$ModEnsenianza)

In [ ]:
busqueda <- filter(notas_uruguay_FIX, Documento <= 48000000 | Documento >= 100000000)
unique(busqueda$Documento)

In [ ]:
#Eliminamos Estudiantes de DNI muy antiguo para estar en nivel primario, sabiendo quee se filtro para tener escuelas comunes
notas_uruguay_FIX <- notas_uruguay_FIX %>%
  filter(Documento >= 48000000 & Documento <= 100000000) #Tambien se eliminan casos extremos, por ej los de dni mayor a medio millon

In [ ]:
uruguay_primero <- filter(notas_uruguay_FIX, Año_cursado==1)
nota_mate<-uruguay_primero %>% group_by(CUE,asignatura) %>%  filter(asignatura=="MATEMATICA")
dim(nota_mate)
nota_mate

In [ ]:
#Matriculas cargada en Notas
for (numero in 1:6) {
  notas_año <- filter(notas_uruguay_FIX, Año_cursado == numero)
  nota_mate <- notas_año %>% filter(asignatura == "MATEMATICA")
  dimensiones <- dim(nota_mate)
  cat("\n","Grado cursado:", numero,"\n")
  cat("Número de filas:", dimensiones[1], "\n")
}

In [ ]:
write_xlsx(notas_uruguay_FIX, "C:/Users/Elias-PC/Facultad/Uruguay_Primaria.xlsx")

In [ ]:
#borramos todo lo que tenemos en memoria
rm(list = ls())
gc()

In [ ]:
library(readxl)
library(dplyr)
library("stringr")
Uruguay_Primaria <- read_excel("C:/Users/Elias-PC/Facultad/Uruguay_Primaria.xlsx")
Uruguay_Primaria$asignatura <- toupper(Uruguay_Primaria$asignatura)
#View(Uruguay_Primaria)

In [ ]:
variables_lengua <- grep("ING", Uruguay_Primaria$asignatura, value = TRUE)
print(unique(variables_lengua))

In [ ]:
print(unique(Uruguay_Primaria$asignatura))

In [ ]:
Uruguay_Primaria$asignatura <- str_replace(Uruguay_Primaria$asignatura,"INGLÉS", "INGLES")
Uruguay_Primaria$asignatura <- str_replace(Uruguay_Primaria$asignatura,"INGLÈS", "INGLES")
Uruguay_Primaria$asignatura <- str_replace(Uruguay_Primaria$asignatura,"IDIOMA-INGLES", "INGLES")

variables <- grep("ING", Uruguay_Primaria$asignatura, value = TRUE)

# Mostrar las variables que contienen "ING"
print(unique(variables))

In [ ]:
# Mostrar las variables que contienen "LENG"
variables <- grep("LENG", Uruguay_Primaria$asignatura, value = TRUE)
print(unique(variables))

# Mostrar las variables que contienen "LIT"
variables <- grep("LIT", Uruguay_Primaria$asignatura, value = TRUE)

print(unique(variables))

In [ ]:
Uruguay_Primaria$asignatura <- str_replace(Uruguay_Primaria$asignatura,"ACOMP. AL ESTUDIO LENGUA Y CS .SOCIALES", "ACOMP. LENGUA Y CS SOC.")
Uruguay_Primaria$asignatura <- str_replace(Uruguay_Primaria$asignatura,"ACOMP LENG Y SOC", "ACOMP. LENGUA Y CS SOC.")
Uruguay_Primaria$asignatura <- str_replace(Uruguay_Primaria$asignatura, "ACOMPAÑAMIENTO AL ESTUDIO LENGUA Y CS SOCIALES","ACOMP. LENGUA Y CS SOC.")

In [ ]:
variables <- grep("MAT", Uruguay_Primaria$asignatura, value = TRUE)
print(unique(variables))

In [ ]:
Uruguay_Primaria$asignatura <- str_replace(Uruguay_Primaria$asignatura,"ACOMP MAT Y NAT", "ACOMP. MAT Y CS. NAT.")
Uruguay_Primaria$asignatura <- str_replace(Uruguay_Primaria$asignatura,"ACOMP. AL ESTUDIO MATEMÁTICA Y CIENCIAS NATURALES", "ACOMP. MAT Y CS. NAT.")
Uruguay_Primaria$asignatura <- str_replace(Uruguay_Primaria$asignatura, "ACOMP AL ESTUDIO MAT Y CS NATURALES","ACOMP. MAT Y CS. NAT.")

In [ ]:
variables <- grep("EDU", Uruguay_Primaria$asignatura, value = TRUE)
print(unique(variables))

In [ ]:
Uruguay_Primaria$asignatura <- str_replace(Uruguay_Primaria$asignatura,"EDUCACIÓN TECNOLÓGICA", "EDUCACION TECNOLOGICA")
Uruguay_Primaria$asignatura <- str_replace(Uruguay_Primaria$asignatura,"EDUCACIÓN FÍSICA", "EDUCACION FISICA")
Uruguay_Primaria$asignatura <- str_replace(Uruguay_Primaria$asignatura,"EDUCACIÓN MUSICAL", "EDUCACION MUSICAL")

variables <- grep("EDU", Uruguay_Primaria$asignatura, value = TRUE)
print(unique(variables))

In [ ]:
variables <- grep("CIEN", Uruguay_Primaria$asignatura, value = TRUE)
print(unique(variables))

In [ ]:
Uruguay_Primaria$asignatura <- str_replace(Uruguay_Primaria$asignatura,"CONSTRUYENDO EL FUTUTO","CONSTRUYENDO EL FUTURO")
print(unique(Uruguay_Primaria$asignatura))

In [ ]:
distribucion_primaria <- Uruguay_Primaria %>%
    distinct(Documento, Año_cursado) %>% # Obtengo los estudiantes unicos (sino sumaba por cada materia en la que estaba anotado)
    count(Año_cursado, name= "cantidad_estudiantes") %>%
    mutate(Porcentaje = cantidad_estudiantes / sum(cantidad_estudiantes) * 100) %>%
    mutate(Porcentaje = round(Porcentaje,2))

distribucion_primaria

In [ ]:
distribucion_primaria <- Uruguay_Primaria %>%
    distinct(Documento, Turno) %>%
    count(Turno, name= "cantidad_estudiantes") %>%
    mutate(Porcentaje = cantidad_estudiantes / sum(cantidad_estudiantes) * 100) %>%
    mutate(Porcentaje = round(Porcentaje,2)) %>%
    add_row(Turno = "TOTAL",
            cantidad_estudiantes = sum(distribucion_primaria$cantidad_estudiantes),
            Porcentaje = sum(distribucion_primaria$Porcentaje))

distribucion_primaria

In [ ]:
tabla_frecuencias <- Uruguay_Primaria %>%
  distinct(Documento, EsPrivada) %>%
  count(EsPrivada, name = "Total") %>%
  mutate(
    Frecuencia_Relativa = round(Total / sum(Total),2),
    Frecuencia_Relativa_Acumulada = cumsum(Frecuencia_Relativa),
    Frecuencia_Porcentual = Frecuencia_Relativa * 100,
    Frecuencia_Porcentual_Acumulada = cumsum(Frecuencia_Porcentual)
  )

tabla_frecuencias

Actividad 4 (LENGUA)

In [ ]:
tablas_frecuencia_lista1 <- list()

for (numero in 1:6) {
    uru_lengua <- filter(Uruguay_Primaria, asignatura == "LENGUA", Año_cursado == numero)
    uru_lengua <- subset(uru_lengua, !(Resultado == "NA"))
    tabla_frecuencias <- uru_lengua %>%
        count(Resultado, name = "Total") %>%
        mutate(
            Frecuencia_Relativa = round(Total / sum(Total), 2),
            Frecuencia_Relativa_Acumulada = cumsum(Frecuencia_Relativa),
            Frecuencia_Porcentual = Frecuencia_Relativa * 100,
            Frecuencia_Porcentual_Acumulada = cumsum(Frecuencia_Porcentual)
        )
    
    tablas_frecuencia_lista1[[numero]] <- tabla_frecuencias
    
    # Calcula la moda de los promedios de calificaciones
    moda <- as.numeric(names(sort(table(uru_lengua$Promedio), decreasing = TRUE)[1]))
    print(moda)
}
tablas_frecuencia_lista1

In [ ]:
tablas_frecuencia_lista <- list()

for (numero in 1:6) {
    uru_lengua <- filter(Uruguay_Primaria, asignatura == "LENGUA", Año_cursado == numero)
    uru_lengua <- subset(uru_lengua, !(Promedio == "NA"))
    tabla_frecuencias <- uru_lengua %>%
        count(Promedio, name = "Total") %>%
        mutate(
            Frecuencia_Relativa = round(Total / sum(Total), 2),
            Frecuencia_Relativa_Acumulada = cumsum(Frecuencia_Relativa),
            Frecuencia_Porcentual = Frecuencia_Relativa * 100,
            Frecuencia_Porcentual_Acumulada = cumsum(Frecuencia_Porcentual)
        )

    
    tablas_frecuencia_lista[[numero]] <- tabla_frecuencias

}

tablas_frecuencia_lista

Actividad 4 (MATEMATICA)

In [ ]:
tablas_frecuencia_lista <- list()

for (numero in 1:6) {
    uru_lengua <- filter(Uruguay_Primaria, asignatura == "MATEMATICA", Año_cursado == numero)
    uru_lengua <- subset(uru_lengua, !(Promedio == "NA"))
    tabla_frecuencias <- uru_lengua %>%
        count(Promedio, name = "Total") %>%
        mutate(
            Frecuencia_Relativa = round(Total / sum(Total), 2),
            Frecuencia_Relativa_Acumulada = cumsum(Frecuencia_Relativa),
            Frecuencia_Porcentual = Frecuencia_Relativa * 100,
            Frecuencia_Porcentual_Acumulada = cumsum(Frecuencia_Porcentual)
        )
    
    tablas_frecuencia_lista[[numero]] <- tabla_frecuencias
}
tablas_frecuencia_lista

In [ ]:
tablas_frecuencia_lista2 <- list()

for (numero in 1:6) {
    uru_lengua <- filter(Uruguay_Primaria, asignatura == "MATEMATICA", Año_cursado == numero)
    uru_lengua <- subset(uru_lengua, !(Resultado == "NA"))
    tabla_frecuencias <- uru_lengua %>%
        count(Resultado, name = "Total") %>%
        mutate(
            Frecuencia_Relativa = round(Total / sum(Total), 2),
            Frecuencia_Porcentual = Frecuencia_Relativa * 100
        )
    tablas_frecuencia_lista2[[numero]] <- tabla_frecuencias
}
tablas_frecuencia_lista2

In [ ]:
for (numero in 1:6) {
    uru_lengua <- filter(Uruguay_Primaria, asignatura == "LENGUA", Año_cursado == numero)
    uru_lengua <- subset(uru_lengua, !(is.na(Promedio)))

    cat("\n","GRADO", numero, "°\n")
    resumen <- round(summary(uru_lengua$Promedio),2)
    print(resumen)

    tabla_frecuencias <- table(uru_lengua$Promedio)
    moda <- as.numeric(names(sort(table(uru_lengua$Promedio), decreasing = TRUE))[1])
    cat("La moda es ->", moda, "\n\n")
    # Calcular el promedio medio
    promedio_medio <- round(mean(uru_lengua$Promedio),2)
    cat("El promedio medio es ->", promedio_medio, "\n\n")

    # Identificar outliers
    Q1 <- quantile(uru_lengua$Promedio, 0.25)
    Q3 <- quantile(uru_lengua$Promedio, 0.75)
    IQR <- Q3 - Q1
    cat("El IQR es ->", IQR, "\n\n")

    #Desvio estandar
    desvio <- round(sd(uru_lengua$Promedio),2)
    cat("El desvio esntadar es ->", desvio, "\n\n")
    
    #outliers
    lim_inf <- Q1 - 1.5 * IQR
    lim_sup <- Q3 + 1.5 * IQR
    outliers <- uru_lengua$Promedio[uru_lengua$Promedio < lim_inf | uru_lengua$Promedio > lim_sup]
    cat("El valor a partir del cual se considera un promedio outlier es:", lim_inf, "\n")
    cat("El valor hasta el cual se considera un promedio outlier es:", lim_sup, "\n")
    
    #Determinamos si son valores raros
    if (length(outliers) > 0) {
        if (length(outliers) == 1) {
            cat("Hay 1 valor atípico.\n")
        } else {
            cat("Hay", length(outliers), "valores extremos.\n")
        }
    } else {
        cat("No hay outliers.\n")
    }

    cuantiles_descartados <- quantile(uru_lengua$Promedio, c(0.25, 0.75), na.rm = TRUE)

    # Calcular los límites del rango intercuartílico (IQR) después de descartar los cuantiles
    valor_inferior_25 <- cuantiles_descartados[1]
    valor_superior_25 <- cuantiles_descartados[2]
    
    # Mostrar los valores entre los cuales se concentra el 50% de los promedios
    cat("Los valores entre los cuales se concentra el 50% de los promedios son:", valor_inferior_25, "y", valor_superior_25, "\n")

    #75% de las notas son menores que
    valor_75 <- quantile(uru_lengua$Promedio, 0.75)
    cat("El 75% de las notas son menores que:", valor_75, "\n")

    #50% de las notas son mayores que
    mediana <- median(uru_lengua$Promedio)
    cat("El 50% de las notas son mayores que:", mediana, "\n")
}

In [ ]:
for (numero in 1:6) {
    uru_lengua <- filter(Uruguay_Primaria, asignatura == "MATEMATICA", Año_cursado == numero)
    uru_lengua <- subset(uru_lengua, !(is.na(Promedio)))

    cat("\n","GRADO", numero, "°\n")
    resumen <- round(summary(uru_lengua$Promedio),2)
    print(resumen)

    tabla_frecuencias <- table(uru_lengua$Promedio)
    moda <- as.numeric(names(sort(table(uru_lengua$Promedio), decreasing = TRUE))[1])
    cat("La moda es ->", moda, "\n\n")
    # Calcular el promedio medio
    promedio_medio <- round(mean(uru_lengua$Promedio),2)
    cat("El promedio medio es ->", promedio_medio, "\n\n")

    # Identificar outliers
    Q1 <- quantile(uru_lengua$Promedio, 0.25)
    Q3 <- quantile(uru_lengua$Promedio, 0.75)
    IQR <- Q3 - Q1
    cat("El IQR es ->", IQR, "\n\n")

    #Desvio estandar
    desvio <- round(sd(uru_lengua$Promedio),2)
    cat("El desvio esntadar es ->", desvio, "\n\n")
    
    #outliers
    lim_inf <- Q1 - 1.5 * IQR
    lim_sup <- Q3 + 1.5 * IQR
    outliers <- uru_lengua$Promedio[uru_lengua$Promedio < lim_inf | uru_lengua$Promedio > lim_sup]
    cat("El valor a partir del cual se considera un promedio outlier es:", lim_inf, "\n")
    cat("El valor hasta el cual se considera un promedio outlier es:", lim_sup, "\n")
    
    #Determinamos si son valores raros
    if (length(outliers) > 0) {
        if (length(outliers) == 1) {
            cat("Hay 1 valor atípico.\n")
        } else {
            cat("Hay", length(outliers), "valores extremos.\n")
        }
    } else {
        cat("No hay outliers.\n")
    }

    cuantiles_descartados <- quantile(uru_lengua$Promedio, c(0.25, 0.75), na.rm = TRUE)

    # Calcular los límites del rango intercuartílico (IQR) después de descartar los cuantiles
    valor_inferior_25 <- cuantiles_descartados[1]
    valor_superior_25 <- cuantiles_descartados[2]
    
    # Mostrar los valores entre los cuales se concentra el 50% de los promedios
    cat("Los valores entre los cuales se concentra el 50% de los promedios son:", valor_inferior_25, "y", valor_superior_25, "\n")

    #75% de las notas son menores que
    valor_75 <- quantile(uru_lengua$Promedio, 0.75)
    cat("El 75% de las notas son menores que:", valor_75, "\n")

    #50% de las notas son mayores que
    mediana <- median(uru_lengua$Promedio)
    cat("El 50% de las notas son mayores que:", mediana, "\n")
}

In [ ]:
library("writexl")
write_xlsx(Uruguay_Primaria, "C:/Users/Elias-PC/Facultad/Uruguay_PrimariaFIX.xlsx")

In [ ]:
#borramos todo lo que tenemos en memoria
rm(list = ls())
gc()

In [ ]:
library(readxl)
library(dplyr)
library(ggplot2)
library(nortest)
library(moments)
library(rlang)
Uruguay_Primaria <- read_excel("C:/Users/Elias-PC/Facultad/Uruguay_PrimariaFIX.xlsx")

In [ ]:
# Function to calculate frequencies
Calcular_frecuencias <- function(DF, columna) { 
  resultado <- DF %>%
    group_by(!!ensym(columna)) %>%
    distinct(Documento, .keep_all = TRUE) %>%  # Ensure distinct works correctly
    summarise(Frecuencia = n()) %>% 
    arrange(Frecuencia)
  return(resultado)
}

# Calculate frequencies
frecuencia_ <- Calcular_frecuencias(Uruguay_Primaria, Año_cursado)

# Convert Año_cursado to a factor to ensure it is treated as a discrete variable
frecuencia_$Año_cursado <- as.factor(frecuencia_$Año_cursado)

In [ ]:
# Plot the histogram of absolute frequencies
ggplot(frecuencia_, aes(x = Año_cursado, y = Frecuencia, fill = Año_cursado)) +
  geom_bar(stat = "identity") +
  labs(title = "Distribución de Estudiantes por Nivel en Primaria",
       x = "Grado Cursado", y = "Número de Estudiantes") +
  scale_fill_brewer(palette = "Set2") +
  theme(legend.position = "none",  # Remove the legend
        axis.text = element_text(size = 15, color = "black"),  # Darken and enlarge axis numbers
        axis.title = element_text(size = 16, color = "black"),
        plot.title = element_text(size = 17, color = "black"),
        panel.border = element_rect(colour = "black", fill = NA, linewidth = 1))  # Add a border to the plot

In [ ]:
#png(filename="D:/Descargas/tu_grafico.png", width=1090, height=1020, res=172) #elejimos la resolucion y otros paramentros
#par(bg = "transparent") #le sacamos el fondo

Uruguay_Primaria_unicos <- Uruguay_Primaria %>%
  distinct(Documento, EsPrivada)
Uruguay_Primaria_unicos$EsPrivada <- ifelse(Uruguay_Primaria_unicos$EsPrivada == 'N', 'Pública', 'Privada')

pie_commute <- round(100 * table(Uruguay_Primaria_unicos$EsPrivada) / length(Uruguay_Primaria_unicos$EsPrivada), 2)

colors = c("#bb3af2",'#4286f4')

pie(pie_commute, labels = paste0(names(pie_commute),'\n', pie_commute, ' %'),
    main = "",
    col = colors,
    radius = .9,
    col.main = "black",
    col.sub = "blue4",
    angle = seq(0,180,35),
    border = 'blue')

# ponemos el titulo mas cerca del grafico cambiando line a 0
mtext(text = "Distribución de Escuelas Públicas y Privadas Nivel Primario", side = 3, line = 0, cex = 1.3)


#dev.off() # Esto cierra el dispositivo de gráficos y guarda el archivo

In [ ]:
Uruguay_Primaria_unicos <- Uruguay_Primaria %>%
  distinct(Documento, Turno)

pie_commute <- round(100 * table(Uruguay_Primaria_unicos$Turno) / length(Uruguay_Primaria_unicos$Turno), 2)

colors <- c('#4286f4', '#bb3af2', '#ed2f52', '#efc023')

pie(pie_commute, 
    labels = paste0(names(pie_commute),'\n', pie_commute, ' %'),
    main = "Distribución de Estudiantes por Turno en Primaria",
    col = colors,
    radius = 0.9,
    col.main = "black",
    col.sub = "blue4", 
    angle = seq(0, 180, 35),
    border = "black")

In [ ]:
#png(filename="D:/Descargas/tu_grafico.png", width=1220, height=1020, res=175)
frecuencia_ <- Calcular_frecuencias(Uruguay_Primaria,Turno)
#Reordenamos por niveles
frecuencia_$Turno <- factor(frecuencia_$Turno, levels = frecuencia_$Turno[order(frecuencia_$Frecuencia)])

colores <- c("#FC8D62", "#8DA0CB", "#E78AC3", "#96D4C0")

ggplot(frecuencia_, aes(x = Turno, y = Frecuencia, fill = colores)) +
  geom_bar(stat = "identity") +
  labs(title = "Distribución de Estudiantes por Turno en Primaria",
       x = "Turno", y = "Cantidad de Estudiantes") +
  scale_fill_brewer(palette = "Set2") +
 geom_text(aes(label = Frecuencia), position = position_stack(vjust = 0.53), size = 8, color = "black", fontface = "bold")+
  theme(legend.position = "none",  # saco la leyenda
        axis.text = element_text(size = 18, color = "black"), #Pongo los numeros y de los ejes mas oscuros y mas grandes
        axis.title = element_text(size = 18, color = "black"),
        axis.text.x = element_text(size = 17, color = "black", face = "bold"),  # Tamaño, color y estilo de los números del eje X
        axis.text.y = element_text(size = 17, color = "black", face = "bold"),
        plot.title = element_text(size = 19, color = "black"),
        panel.border = element_rect(colour = "black", fill=NA, size=1), #le agrego un borde al grafico
        panel.grid.major = element_line(colour = "black"), #lineas del grid
        panel.grid.minor = element_line(colour = "black"))
#dev.off()

In [ ]:
#png(filename="D:/Descargas/tu_grafico.png", width=1140, height=1020, res=175) #para guardar el graf

# Calcular las frecuencias
frecuencia_ <- Calcular_frecuencias(Uruguay_Primaria, Año_cursado)

# Identificar la frecuencia máxima y mínima
max_freq <- max(frecuencia_$Frecuencia)
min_freq <- min(frecuencia_$Frecuencia)

# Crear una nueva columna para los colores
frecuencia_$Color <- ifelse(frecuencia_$Frecuencia == max_freq, "red", ifelse(frecuencia_$Frecuencia == min_freq, "blue", "gray"))

# Graficar el gráfico de barras con ggplot2
ggplot(frecuencia_, aes(x = Año_cursado, y = Frecuencia, fill = Color)) +
  geom_col() +
  scale_fill_manual(values = c("red" = "#00A200", "blue" = "#DB2514", "gray" = "gray")) +
  labs(title = "Distribución de Estudiantes por Nivel en Primaria",
       x = "Grado Cursado", y = "Cantidad de Estudiantes") +
  geom_text(aes(label = Frecuencia), position = position_stack(vjust = 0.5), size = 8, color = "black", fontface = "bold")+
  theme(legend.position = "none",  # saco la leyenda
        axis.text = element_text(size = 18, color = "black"), #Pongo los numeros y de los ejes mas oscuros y mas grandes
        axis.title = element_text(size = 18, color = "black"),
        axis.text.x = element_text(size = 17, color = "black", face = "bold"),  # Tamaño, color y estilo de los números del eje X
        axis.text.y = element_text(size = 17, color = "black", face = "bold"),
        plot.title = element_text(size = 18, color = "black"),
        panel.border = element_rect(colour = "black", fill=NA, size=1), #le agrego un borde al grafico
        panel.grid.major = element_line(colour = "black"), #lineas del grid
        panel.grid.minor = element_line(colour = "black"))
#dev.off()

In [ ]:
Uruguay_Primaria_unicos <- Uruguay_Primaria %>%
  distinct(Documento, Resultado, asignatura, Año_cursado)

for (numero in 1:6) {
    uru_lengua <- filter(Uruguay_Primaria_unicos, asignatura == "LENGUA", Año_cursado == numero)
    uru_lengua <- subset(uru_lengua, !(Resultado == "NA"))
    pie_commute <- round(100 * table(uru_lengua$Resultado) / length(uru_lengua$Resultado),2)
    
    colors = c('#4286f4','#bb3af2')
    
    pie(pie_commute, labels = paste0(names(pie_commute),'\n', pie_commute, ' %'),
        main = paste(""),
        col = colors,
        radius = .9,
        col.main = "black",
        col.sub = "blue4",
        angle = seq(0,180,35),
        border = 'blue')
    # ponemos el titulo mas cerca del grafico cambiando line a 0
    mtext(text = paste("Distribución Resultados de Lengua en Primaria", numero, "Grado"), side = 3, line = 0, cex = 1.5)


}
#png(filename="D:/Descargas/tu_grafico.png", width=1060, height=1020, res=172) #elejimos la resolucion y otros paramentros
#par(bg = "transparent") #le sacamos el fondo

#GRAFICO TOMANDO EN CUENTA TODOS LOS AÑOS
Uruguay_Primaria_unicos <- Uruguay_Primaria %>%
  distinct(Documento, Resultado, asignatura)

uru_lengua <- filter(Uruguay_Primaria_unicos, asignatura == "LENGUA")
uru_lengua <- subset(uru_lengua, !(Resultado == "NA"))
pie_commute <- round(100 * table(uru_lengua$Resultado) / length(uru_lengua$Resultado),2)

colors = c('#4286f4','#bb3af2')

pie(pie_commute, labels = paste0(names(pie_commute),'\n', pie_commute, ' %'),
    main = paste(""),
    col = colors,
    radius = .9,
    col.main = "black",
    col.sub = "blue4",
    angle = seq(0,180,35),
    border = 'blue')


# ponemos el titulo mas cerca del grafico cambiando line a 0
mtext(text = "Distribución Resultado general en Lengua, Primaria", side = 3, line = 0, cex = 1.5)

#dev.off()

In [ ]:
Uruguay_Primaria_unicos <- Uruguay_Primaria %>%
  distinct(Documento, Resultado, asignatura, Año_cursado)

for (numero in 1:6) {
    uru_lengua <- filter(Uruguay_Primaria_unicos, asignatura == "MATEMATICA", Año_cursado == numero)
    uru_lengua <- subset(uru_lengua, !(Resultado == "NA"))
    pie_commute <- round(100 * table(uru_lengua$Resultado) / length(uru_lengua$Resultado),2)

    colors = c('#4286f4','#bb3af2')
    
    pie(pie_commute, labels = paste0(names(pie_commute),'\n', pie_commute, ' %'),
        main = paste(""),
        col = colors,
        radius = .9,
        col.main = "black",
        col.sub = "blue4",
        angle = seq(0,180,35),
        border = 'blue')

    # ponemos el titulo mas cerca del grafico cambiando line a 0
    mtext(text = paste("Distribución Resultados de Matematica en Primaria", numero, "Grado"), side = 3, line = 0, cex = 1.5)
    
}
#GRAFICO TOMANDO EN CUENTA TODOS LOS AÑOS
#png(filename="D:/Descargas/tu_grafico.png", width=1180, height=1080, res=172) #elejimos la resolucion y otros paramentros
par(bg = "transparent") #le sacamos el fondo

Uruguay_Primaria_unicos <- Uruguay_Primaria %>%
  distinct(Documento, Resultado, asignatura)

uru_lengua <- filter(Uruguay_Primaria_unicos, asignatura == "MATEMATICA")
uru_lengua <- subset(uru_lengua, !(Resultado == "NA"))
pie_commute <- round(100 * table(uru_lengua$Resultado) / length(uru_lengua$Resultado),2)

colors = c('#4286f4','#bb3af2')

pie(pie_commute, labels = paste0(names(pie_commute),'\n', pie_commute, ' %'),
    main = paste(""),
    col = colors,
    radius = .9,
    col.main = "black",
    col.sub = "blue4",
    angle = seq(0,180,35),
    border = 'blue')

# ponemos el titulo mas cerca del grafico cambiando line a 0
mtext(text = "Distribución Resultado general en Matematica, Primaria", side = 3, line = 0, cex = 1.5)

#dev.off()

In [ ]:
for (i in 1:6) {
  # Filtrar los datos por el año correspondiente y la materia de Matematica
  datos_año_matematica <- Uruguay_Primaria[Uruguay_Primaria$Año_cursado == i & Uruguay_Primaria$asignatura == "MATEMATICA", ]

  datos_año_matematica <- subset(datos_año_matematica, !(Resultado == "NA"))
  valores <- datos_año_matematica$Promedio

   if (length(valores) < 50) {
      #test de shapiro para verf normalidad
      qqnorm(valores, main = paste("PRIMARIA: Q-Q Plot - ", i, "Grado - Matematica"))
      print("teste de shapiro")
      aux <-shapiro.test(valores)
      print(aux)
      qqline(valores, col = "red")  #Linea del QQPLOT
    }
    
    else{

      qqnorm(valores, main = paste("PRIMARIA: Q-Q Plot - Grado", i, "- Matematica"))
      qqline(valores, col = "red")

      #TEST de ks y lillie
      suppressWarnings({ #ignoramos los warning
      ks_test_result <- ks.test(valores, "pnorm")
      print(ks_test_result)
      aux <- lillie.test(valores)
      print(aux)
    })
    }
}

In [ ]:
resultados_por_año <- list()

# Realizar el análisis para cada grado
for (i in 1:6) {
  # Filtrar los datos por el grado correspondiente
  datos_año <- Uruguay_Primaria[Uruguay_Primaria$Año_cursado == i & Uruguay_Primaria$asignatura == "MATEMATICA", ]
  datos_año <- subset(datos_año, !(is.na(Promedio)))
  
  # Calcular el rango de variación de los datos
  rango <- range(datos_año$Promedio)
  
  # Determinar si la distribución es simétrica y el tipo de asimetría
  coeficiente_asimetria <- skewness(datos_año$Promedio)
  simetria <- ifelse(coeficiente_asimetria == 0, "simétrica",
                     ifelse(coeficiente_asimetria > 0, "asimétrica a la derecha",
                            "asimétrica a la izquierda"))
  
  # Calcular la media
  media <- round(mean(datos_año$Promedio),2)
  
  # Calcular la mediana
  mediana <- median(datos_año$Promedio)
  
  # Calcular el rango intercuartílico
  rango_intercuartilico <- IQR(datos_año$Promedio)
  
  # Calcular los cuartiles
  cuartiles <- quantile(datos_año$Promedio, c(0.25, 0.5, 0.75))
  
  # Calcular el límite superior de los valores atípicos
  limite_superior_atipicos <- cuartiles[3] + 1.5 * rango_intercuartilico
  
  # Contar la cantidad de datos atípicos
  datos_atipicos <- datos_año$Promedio[datos_año$Promedio > limite_superior_atipicos]
  
  # Evaluar si el rendimiento fue óptimo
  rendimiento_optimo <- ifelse(mediana >= 6, "Sí", "No")
  
  # Almacenar los resultados en la lista
  resultados_por_año[[paste("Año", i)]] <- list(
    Rango = rango,
    Simetría = simetria,
    Media = media,
    Mediana = mediana,
    RangoIntercuartilico = rango_intercuartilico,
    Cuartiles = cuartiles,
    LimiteSuperiorAtipicos = limite_superior_atipicos,
    CantidadDatosAtipicos = length(datos_atipicos),
    RendimientoOptimo = rendimiento_optimo
  )
}

for (i in 1:6) {
  cat("Resultados para el año", i, ":\n")
  cat("  Rango de variación de los datos (Máximo - Mínimo):", resultados_por_año[[paste("Año", i)]]$Rango[2], "-", resultados_por_año[[paste("Año", i)]]$Rango[1], "=", resultados_por_año[[paste("Año", i)]]$Rango[2] - resultados_por_año[[paste("Año", i)]]$Rango[1], "\n")
  cat("  La distribución es", resultados_por_año[[paste("Año", i)]]$Simetría, "\n")
  cat("  Media de los datos:", resultados_por_año[[paste("Año", i)]]$Media, "\n")
  cat("  Mediana de los datos:", resultados_por_año[[paste("Año", i)]]$Mediana, "\n")
  cat("  Rango intercuartílico:", resultados_por_año[[paste("Año", i)]]$RangoIntercuartilico, "\n")
  cat("  Cuartiles:", resultados_por_año[[paste("Año", i)]]$Cuartiles, "\n")
  cat("  Límite superior para considerar valores atípicos:", resultados_por_año[[paste("Año", i)]]$LimiteSuperiorAtipicos, "\n")
  cat("  Cantidad de datos atípicos:", resultados_por_año[[paste("Año", i)]]$CantidadDatosAtipicos, "\n")
  cat("  ¿El rendimiento fue óptimo?:", resultados_por_año[[paste("Año", i)]]$RendimientoOptimo, "\n\n")
}

In [ ]:
for (numero in 1:6) {
    uru_mate <- filter(Uruguay_Primaria, asignatura == "MATEMATICA", Año_cursado == numero)
    boxplot(uru_mate$Promedio, main = paste("Boxplot de Promedio Matematica GRADO", numero), ylab = "Promedio")
    cat("\n", "GRADO", numero, "°\n")
}

In [ ]:
for (numero in 1:6) {
    uru_mate <- filter(Uruguay_Primaria, asignatura == "MATEMATICA", Año_cursado == numero)
    
    datos_promedio <- na.omit(uru_mate$Promedio) #Borramos los NA
    
    # Estimacion kernel de la densidad
    d <- density(datos_promedio)
    
    # Grafico de la densidad
    plot(d, lwd = 2, main = paste("Grafico densidad kernel de Promedio - GRADO", numero,"° Matematica"), xlab = "Promedios", ylab = "Densidad")
    cat("\n", "GRADO", numero, "°\n")
}

In [ ]:
for (numero in 1:6) {
    datos_año_lengua <- filter(Uruguay_Primaria, Año_cursado == numero & asignatura == "MATEMATICA")
    datos_año_lengua <- subset(datos_año_lengua, !(is.na(Promedio)))

    aux <- ggplot(datos_año_lengua, aes(x = Promedio)) +
        geom_histogram(binwidth = 1, fill = "skyblue", color = "black") + # con binwitch se elige el "intervalo"
        labs(title = paste("Grafico -", numero, "Grado - Matematica"),
             x = "Promedio",
             y = "Frecuencia")
    print(aux)
    
    #MODA
    notas <- datos_año_lengua$Promedio
    tabla_frecuencias <- table(notas)
    modas <- names(tabla_frecuencias[tabla_frecuencias == max(tabla_frecuencias)])
    
    # Imprimir el valor de la moda
    cat("Valor(es) de la moda:", modas, "\n")
    
    if (length(modas)== 1) {
      cat("La distribucion es unimodal", numero, "Grado - Matematica")
    } else {
      cat("La distribucion es multimodal", numero, "Grado - Matematica")
    }
    
    #Coeficiente de Fisher
    coef_fisher <- sum((datos_año_lengua$Promedio - mean(datos_año_lengua$Promedio))^3) / (length(datos_año_lengua$Promedio) * sd(datos_año_lengua$Promedio)^3/length(datos_año_lengua$Promedio))
    
    cat("\n","Coeficiente de Asimetría de Fisher:", coef_fisher, "\n")
    
    #prints de Fisher
    if (coef_fisher > 0) {
      cat("\n","La distribución es asimétrica positiva (sesgada hacia la derecha)","\n")
    } else if (coef_fisher < 0) { #En R no existe elif
      cat("\n","La distribución es asimétrica negativa (sesgada hacia la izquierda)","\n")
    } else {
      cat("\n","La distribución es simétrica","\n")
    }
}

In [ ]:
resultados_por_año <- list()

# Realizar el análisis para cada grado
for (i in 1:6) {
  # Filtrar los datos por el grado correspondiente
  datos_año <- Uruguay_Primaria[Uruguay_Primaria$Año_cursado == i & Uruguay_Primaria$asignatura == "LENGUA", ]
  datos_año <- subset(datos_año, !(is.na(Promedio)))
  
  # Calcular el rango de variación de los datos
  rango <- range(datos_año$Promedio)
  
  # Determinar si la distribución es simétrica y el tipo de asimetría
  coeficiente_asimetria <- skewness(datos_año$Promedio)
  simetria <- ifelse(coeficiente_asimetria == 0, "simétrica",
                     ifelse(coeficiente_asimetria > 0, "asimétrica a la derecha",
                            "asimétrica a la izquierda"))
  
  # Calcular la media
  media <- round(mean(datos_año$Promedio),2)
  
  # Calcular la mediana
  mediana <- median(datos_año$Promedio)
  
  # Calcular el rango intercuartílico
  rango_intercuartilico <- IQR(datos_año$Promedio)
  
  # Calcular los cuartiles
  cuartiles <- quantile(datos_año$Promedio, c(0.25, 0.5, 0.75))
  
  # Calcular el límite superior de los valores atípicos
  limite_superior_atipicos <- cuartiles[3] + 1.5 * rango_intercuartilico
  
  # Contar la cantidad de datos atípicos
  datos_atipicos <- datos_año$Promedio[datos_año$Promedio > limite_superior_atipicos]
  
  # Evaluar si el rendimiento fue óptimo
  rendimiento_optimo <- ifelse(mediana >= 6, "Sí", "No")
  
  # Almacenar los resultados en la lista
  resultados_por_año[[paste("Año", i)]] <- list(
    Rango = rango,
    Simetría = simetria,
    Media = media,
    Mediana = mediana,
    RangoIntercuartilico = rango_intercuartilico,
    Cuartiles = cuartiles,
    LimiteSuperiorAtipicos = limite_superior_atipicos,
    CantidadDatosAtipicos = length(datos_atipicos),
    RendimientoOptimo = rendimiento_optimo
  )
}
for (i in 1:6) {
  cat("Resultados para el año", i, ":\n")
  cat("  Rango de variación de los datos (Máximo - Mínimo):", resultados_por_año[[paste("Año", i)]]$Rango[2], "-", resultados_por_año[[paste("Año", i)]]$Rango[1], "=", resultados_por_año[[paste("Año", i)]]$Rango[2] - resultados_por_año[[paste("Año", i)]]$Rango[1], "\n")
  cat("  La distribución es", resultados_por_año[[paste("Año", i)]]$Simetría, "\n")
  cat("  Media de los datos:", resultados_por_año[[paste("Año", i)]]$Media, "\n")
  cat("  Mediana de los datos:", resultados_por_año[[paste("Año", i)]]$Mediana, "\n")
  cat("  Rango intercuartílico:", resultados_por_año[[paste("Año", i)]]$RangoIntercuartilico, "\n")
  cat("  Cuartiles:", resultados_por_año[[paste("Año", i)]]$Cuartiles, "\n")
  cat("  Límite superior para considerar valores atípicos:", resultados_por_año[[paste("Año", i)]]$LimiteSuperiorAtipicos, "\n")
  cat("  Cantidad de datos atípicos:", resultados_por_año[[paste("Año", i)]]$CantidadDatosAtipicos, "\n")
  cat("  ¿El rendimiento fue óptimo?:", resultados_por_año[[paste("Año", i)]]$RendimientoOptimo, "\n\n")
}

In [ ]:
for (numero in 1:6) {
    uru_lengua <- filter(Uruguay_Primaria, asignatura == "LENGUA", Año_cursado == numero)
    boxplot(uru_lengua$Promedio, main= paste("Boxplot de Promedio Lengua GRADO", numero), ylab="Promedio")
    cat("\n","GRADO", numero, "°\n")
}

In [ ]:
for (numero in 1:6) {
    uru_lengua <- filter(Uruguay_Primaria, asignatura == "LENGUA", Año_cursado == numero)
    
    datos_promedio <- na.omit(uru_lengua$Promedio) #Borramos los NA
    
    # Estimacion kernel de la densidad
    d <- density(datos_promedio)
    
    # Grafico de la densidad
    plot(d, lwd = 2, main = paste("Grafico densidad kernel de Promedio - GRADO", numero,"° Lengua"), xlab = "Promedios", ylab = "Densidad")
    cat("\n", "GRADO", numero, "°\n")
}

In [ ]:
for (i in 1:6) {
  # Filtrar los datos por el grado correspondiente y la materia de Matematica
  datos_año_lengua <- Uruguay_Primaria[Uruguay_Primaria$Año_cursado == i & Uruguay_Primaria$asignatura == "LENGUA", ]

  datos_año_lengua <- subset(datos_año_lengua, !(Resultado == "NA"))
  valores <- datos_año_lengua$Promedio


   if (length(valores) < 50) {
      #test de shapiro para verf normalidad
      qqnorm(valores, main = paste("PRIMARIA: Q-Q Plot - ", i, "Grado - Lengua"))
      print("teste de shapiro")
      aux <-shapiro.test(valores)
      print(aux)
      qqline(valores, col = "red")  #Linea del QQPLOT
    }
    else{
      valores_ordenados <- sort(valores)

      qqnorm(valores_ordenados, main = paste("PRIMARIA: Q-Q Plot - Grado", i, "- Lengua"))
      qqline(valores_ordenados, col = "red")

      #TEST de ks
      suppressWarnings({ #ignoramos las warning
      ks_test_result <- ks.test(valores, "pnorm")
      print(ks_test_result)
      aux <- lillie.test(valores)
      print(aux)
    })
    }
}

In [ ]:
for (numero in 1:6) {
    datos_año_lengua <- filter(Uruguay_Primaria, Año_cursado == numero & asignatura == "LENGUA")
    datos_año_lengua <- subset(datos_año_lengua, !(is.na(Promedio)))

    aux <- ggplot(datos_año_lengua, aes(x = Promedio)) +
        geom_histogram(binwidth = 1, fill = "skyblue", color = "black") + # con binwitch se elige el "intervalo"
        labs(title = paste("Grafico -", numero, "Grado - Lengua"),
             x = "Promedio",
             y = "Frecuencia")
    print(aux)

    #MODA
    notas <- datos_año_lengua$Promedio
    tabla_frecuencias <- table(notas)
    modas <- names(tabla_frecuencias[tabla_frecuencias == max(tabla_frecuencias)])
    
    # Imprimir el valor de la moda
    cat("Valor(es) de la moda:", modas, "\n")
    
    if (length(modas)== 1) {
      cat("La distribucion es unimodal", numero, "Grado - Lengua")
    } else {
      cat("La distribucion es multimodal", numero, "Grado - Lengua")
    }
    
    #Coeficiente de Fisher
    coef_fisher <- sum((datos_año_lengua$Promedio - mean(datos_año_lengua$Promedio))^3) / (length(datos_año_lengua$Promedio) * sd(datos_año_lengua$Promedio)^3/length(datos_año_lengua$Promedio))
    
    cat("\n","Coeficiente de Asimetría de Fisher:", coef_fisher, "\n")
    
    #prints de Fisher
    if (coef_fisher > 0) {
      cat("\n","La distribucion es asimetrica positiva (sesgada hacia la derecha)","\n")
    } else if (coef_fisher < 0) { #En R no existe elif
      cat("\n","La distribucion es asimetrica negativa (sesgada hacia la izquierda)","\n")
    } else {
      cat("\n","La distribucion es simetrica","\n")
    }
}

In [ ]:
#borramos todo lo que tenemos en memoria
rm(list = ls())
gc()

In [ ]:
library(readxl)
library(ggplot2)
library(dplyr)
library(psych)
library(tidyr)
library(corrplot)
library(ppcor)
library(ggcorrplot)
library(plotly)


Uruguay_Primaria <- read_excel("C:/Users/Elias-PC/Facultad/Uruguay_PrimariaFIX.xlsx")

In [ ]:
mate_primero <- filter(Uruguay_Primaria, asignatura == "MATEMATICA", Año_cursado =="1")

estudiantes_por_escuela <- mate_primero %>%
  group_by(CUE) %>%
  summarise(Documento = n())

escuelas_con_5_o_mas <- estudiantes_por_escuela %>%
  filter(Documento >= 5) #Yo analice las escuelas con mas de 5 alumnos o algo asi para el informe

escuelas_con_5_o_mas <- escuelas_con_5_o_mas %>% 
  arrange(Documento)

escuelas_con_5_o_mas <- escuelas_con_5_o_mas %>%
  slice(1)

escuela_mas_estudiantes <- estudiantes_por_escuela %>%
  slice_max(order_by = Documento)

View(escuelas_con_5_o_mas)
View(escuela_mas_estudiantes)

In [ ]:
mate_primero_menos <- filter(mate_primero, CUE == 9000567) #Ponemos el CUE de la escuela con menos alumnos

mate_primero_mas <- filter(mate_primero, CUE ==9001410)

ggplot(mate_primero_menos, aes(x = Promedio)) +
  geom_histogram(binwidth = 1, fill = "#87CEFA", color = "black") +
  labs(title = "Histograma de Frecuencias Absolutas - Escuela con menos estudiantes",
       x = "Notas", y = "Frecuencia Absoluta")

# Histograma de frecuencias absolutas para la escuela con mas estudiantes
ggplot(mate_primero_mas, aes(x = Promedio)) +
  geom_histogram(binwidth = 1, fill = "#87CEFA", color = "black") +
  labs(title = "Histograma de Frecuencias Absolutas - Escuela con mas estudiantes",
       x = "Notas", y = "Frecuencia Absoluta+")

In [ ]:
#Tipificamos los datos:
Uruguay_Primaria$Promedio_tipificado <- scale(Uruguay_Primaria$Promedio)

Uruguay_Primaria$Promedio_tipificado

In [ ]:
for (numero in 1:6) {
    uru_mate <- filter(Uruguay_Primaria, asignatura == "MATEMATICA", Año_cursado == numero)
    
    datos_promedio <- na.omit(uru_mate$Promedio_tipificado) #Borramos los NA
    
    # Estimacion kernel de la densidad
    d <- density(datos_promedio)

    # Grafico de la densidad
    plot(d, lwd = 2, main = paste("Grafico densidad kernel de Promedio - GRADO", numero,"° Matematica"), xlab = "Promedio_tipificado", ylab = "Densidad")
    cat("\n", "GRADO", numero, "°\n")
}

In [ ]:
for (numero in 1:6) {
    uru_lengu <- filter(Uruguay_Primaria, asignatura == "LENGUA", Año_cursado == numero)
    
    datos_promedio <- na.omit(uru_lengu$Promedio_tipificado) #Borramos los NA
    
    # Estimacion kernel de la densidad
    d <- density(datos_promedio)

    # Grafico de la densidad
    plot(d, lwd = 2, main = paste("Grafico densidad kernel de Promedio - GRADO", numero,"° Lengua"), xlab = "Promedio_tipificado", ylab = "Densidad")
    cat("\n", "GRADO", numero, "°\n")
}

In [ ]:
datos_estandarizados <- scale(Uruguay_Primaria$Promedio)

# Agregar la variable estandarizada al conjunto de datos original
Uruguay_Primaria$Promedio_estandarizado <- datos_estandarizados

# Estandarizar las variables de promedio de notas en Matemáticas y Lengua
promedio_mat_estandarizado <- scale(Uruguay_Primaria$Promedio_estandarizado[Uruguay_Primaria$Año_cursado == 1 & Uruguay_Primaria$asignatura == "MATEMATICA"])
promedio_lengua_estandarizado <- scale(Uruguay_Primaria$Promedio_estandarizado[Uruguay_Primaria$Año_cursado == 1 & Uruguay_Primaria$asignatura == "LENGUA"])

# Análisis de la información adicional proporcionada por la estandarización
summary(promedio_mat_estandarizado)
summary(promedio_lengua_estandarizado)

In [ ]:
colores <- c("MATEMATICA" = "#FF6347", "LENGUA" = "#00CD66")

for (numero in 1:6) {
  uru_mate <- filter(Uruguay_Primaria, asignatura == "MATEMATICA", Año_cursado == numero)
  uru_lengua <- filter(Uruguay_Primaria, asignatura == "LENGUA", Año_cursado == numero)
  
  boxplot(uru_mate$Promedio, uru_lengua$Promedio, names = c("MATEMATICA", "LENGUA"), 
          main = paste("Boxplot Comparativo Matematica-Lengua", numero), ylab = "Valores", xlab = "Grupos",
          col = colores)
}

In [ ]:
alg_materias_primaria <- filter(Uruguay_Primaria, (asignatura == "LENGUA" | asignatura== "MATEMATICA"))

alg_materias_primaria <- subset(alg_materias_primaria, !(is.na(Promedio))) #eliminamos los promedios que son NA
                                                                           #por culpa de los EPAC.
colores <- c("#1f77b4", "#ff7f0e")

figurita <- plot_ly(alg_materias_primaria, x = ~Año_cursado, y = ~Promedio, color = ~asignatura, type = "box", colors=colores )
figurita <- figurita %>% layout(title = "Grafico interactivo: boxplot agrupado de materias por grado de cursado nivel primario",
                      boxmode = "group",
                      xaxis = list(title = "Grado cursado"),
                      yaxis = list(gridcolor = "grey"))
figurita

In [ ]:
alg_materias_primaria <- filter(Uruguay_Primaria, (asignatura == "LENGUA" | asignatura== "MATEMATICA" 
                                         | asignatura =="CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES"))

alg_materias_primaria <- subset(alg_materias_primaria, !(is.na(Promedio))) #eliminamos los promedios que son NA
                                                                           #por culpa de los EPAC.
colores <- c("#1f77b4", "#ff7f0e")

figurita <- plot_ly(alg_materias_primaria, x = ~Año_cursado, y = ~Promedio, color = ~asignatura, type = "box", colors=colores )
figurita <- figurita %>% layout(title = "Grafico interactivo: boxplot agrupado de materias por grado de cursado nivel primario",
                      boxmode = "group",
                      xaxis = list(title = "Grado cursado"),
                      yaxis = list(gridcolor = "grey"))
figurita

In [ ]:
matematica_manana <- filter(Uruguay_Primaria, asignatura == "MATEMATICA" & Turno == "MAÑANA")
matematica_manana <- subset(matematica_manana, !(is.na(Promedio)))
matematica_tarde <- filter(Uruguay_Primaria, asignatura == "MATEMATICA" & Turno == "TARDE")
matematica_tarde <- subset(matematica_tarde, !(is.na(Promedio)))


ggplot(mapping = aes(x = factor(Año_cursado), y = Promedio, fill = Turno)) +
  geom_boxplot(data = rbind(matematica_manana, matematica_tarde), alpha = 0.8) +
  labs(title = "Comparación de Notas de Matemática por Turno",
       x = "Grado cursado",
       y = "Promedio",
       fill = "Turno")

In [ ]:
# Crear una lista para almacenar los coeficientes de correlación
correlation_coefficients <- list()

for (i in 1:6) {
  # Filtrar los datos para Matemáticas y Educación Física en el grado actual del ciclo
  datos_matematica <- filter(Uruguay_Primaria, asignatura == "MATEMATICA" & Año_cursado == i)
    datos_matematica <- subset(datos_matematica, !(is.na(Promedio)))
  datos_EdFisica <- filter(Uruguay_Primaria, asignatura == "EDUCACION FISICA" & Año_cursado == i)
    datos_EdFisica <- subset(datos_EdFisica, !(is.na(Promedio)))
  
  # Suprimir warnings relacionados con inner_join
  suppressWarnings({
    # Unir los datos de Matemáticas y Educación Física por el documento
    df <- inner_join(datos_matematica, datos_EdFisica, by = "Documento")
  })
  
  # Renombrar las columnas de promedio
  df <- rename(df, Prom_Mat = Promedio.x, Prom_EdFisica = Promedio.y)
  
  # Crear el gráfico de dispersión
  scatter_plot <- ggplot(data = df, aes(x = Prom_Mat, y = Prom_EdFisica)) +
    geom_point() +
    labs(title = paste("Diagrama de Dispersión - Año", i),
         x = "Promedio Matemática",
         y = "Promedio Educación Física") +
    theme_minimal()
  
  # Imprimir el gráfico de dispersión
  print(scatter_plot)
  
    correlation_spearman <- cor(df$Prom_Mat, df$Prom_EdFisica, method = "spearman")
    # Almacenar el coeficiente de correlación de Spearman en la lista
    correlation_coefficients[[i]] <- c(spearman = correlation_spearman)
    # Imprimir el coeficiente de correlación de Spearman
    cat("Año", i, "- Coeficiente de correlación de Spearman:", correlation_spearman, "\n")
}

In [ ]:
# Crear una lista para almacenar los gráficos
scatter_plots <- list()

# Crear una lista para almacenar los coeficientes de correlación
correlation_coefficients <- list()

for (i in 1:6) {
  # Filtrar los datos para Matemáticas y Lengua en el grado actual del ciclo
  datos_matematica <- filter(Uruguay_Primaria, asignatura == "MATEMATICA" & Año_cursado == i)
    datos_matematica <- subset(datos_matematica, !(is.na(Promedio)))
  datos_lengua <- filter(Uruguay_Primaria, asignatura == "LENGUA" & Año_cursado == i)
    datos_lengua <- subset(datos_lengua, !(is.na(Promedio)))
  
  # Suprimir warnings relacionados con inner_join
  suppressWarnings({
    # Unir los datos de Matemáticas y Lengua por el documento
    df <- inner_join(datos_matematica, datos_lengua, by = "Documento")
  })
  
  # Renombrar las columnas de promedio
  df <- rename(df, Prom_Mat = Promedio.x, Prom_Leng = Promedio.y)
  
  # Crear el gráfico de dispersión
  scatter_plot <- ggplot(data = df, aes(x = Prom_Mat, y = Prom_Leng)) +
    geom_point() +
    labs(title = paste("Diagrama de Dispersión - Año", i),
         x = "Promedio Matemática",
         y = "Promedio Lengua") +
    theme_minimal()
  
  # Guardar el gráfico en la lista
  scatter_plots[[i]] <- scatter_plot
  
  # Imprimir el gráfico
  print(scatter_plot)

    correlation_spearman <- suppressWarnings(cor(df$Prom_Mat, df$Prom_Leng, method = "spearman"))
    # Almacenar el coeficiente de correlación de Spearman en la lista
    correlation_coefficients[[i]] <- list(method = "Spearman", value = correlation_spearman)
    # Determinar la fuerza de la asociación para Spearman
    strength <- ifelse(abs(correlation_spearman) < 0.4, "débil",
                       ifelse(abs(correlation_spearman) < 0.7, "moderada", "fuerte"))
    # Imprimir el coeficiente de correlación de Spearman y su fuerza
    cat("Año", i, "- Coeficiente de correlación de Spearman:", correlation_spearman, "- Asociación:", strength, "\n")
 
}

In [ ]:
# Definir las materias de interés
materias <- c("MATEMATICA", "CIENCIAS NATURALES", "CIENCIAS SOCIALES", "EDUCACION FISICA", "LENGUA")

# Crear una lista para almacenar las matrices de correlación
resultados <- list()
resultados_parciales <- list()  # Inicializar lista para resultados parciales

for (anio in 1:6) {
  # Filtrar los datos para el año cursado actual y las materias de interés
  df_materia <- suppressWarnings({
    Uruguay_Primaria %>%
      filter(Año_cursado == as.character(anio) & asignatura %in% materias) %>%
      group_by(Documento, asignatura) %>%
      summarise(Promedio = mean(Promedio), .groups = "drop") %>%  # Suprimir el warning de summarise
      spread(key = asignatura, value = Promedio) %>%
      rename_all(~paste("Prom_", ., sep = "")) %>%
      na.omit()
  })
  
  # Seleccionar solo las columnas de interés
  dataM <- df_materia[, c("Prom_MATEMATICA", "Prom_CIENCIAS NATURALES", "Prom_CIENCIAS SOCIALES", "Prom_LENGUA", "Prom_EDUCACION FISICA")]

  # Transformar los datos a formato numérico
  dataM[] <- lapply(dataM, as.numeric)
  
  # Calcular la matriz de correlación de Spearman
  Scatter_Matrix_spearman <- suppressWarnings({
    pairs.panels(dataM,
                 method = "spearman",
                 ellipses = TRUE,
                 main = paste("Matriz de dispersión de Spearman para el grado", anio))  # Agregar título del año
  })
  
  # Almacenar la matriz de dispersión de Spearman en la lista de resultados
  resultados[[paste("Anio", anio, sep = "_")]] <- Scatter_Matrix_spearman
  
  # Calcular la matriz de correlación parcial
  correlacion_parcial <- pcor(dataM, method = "spearman")
  
  # Almacenar la matriz de correlación parcial en la lista de resultados parciales
  resultados_parciales[[paste("Anio", anio, sep = "_")]] <- correlacion_parcial$estimate
}

# Imprimir los resultados de las matrices de correlación parcial
for (i in seq_along(resultados_parciales)) {
  cat("Matriz de correlación parcial para el grado", i, ":\n")
  print(resultados_parciales[[i]])
  cat("\n")
}

for (numero in 1:6) {
  partial_corr <- resultados_parciales[[numero]]
  aux<- ggcorrplot(partial_corr, hc.order = TRUE, lab = TRUE) + #graficamos la matris de correlacion parcial
    ggtitle(paste("Matriz de correlación parcial", numero,"°grado PRIMARIA")) #le ponemos titulo
  print(aux)
}

In [ ]:
#borramos todo lo que tenemos en memoria
rm(list = ls())
gc()

In [ ]:
library(readxl)
library(dplyr)
library(pwr)
library(stats)
library(car)
library(ggplot2)
library(tidyr)
library("DescTools")
library(reshape2)
library(plotly)
library("aplpack")

Primaria_por_dep <- read.csv("C:/Users/Elias-PC/Facultad/primaria_pordep_fix.csv", encoding = "UTF-8")
Primaria_por_dep

Uruguay_Primaria <- read_excel("C:/Users/Elias-PC/Facultad/Uruguay_PrimariaFIX.xlsx")

#filtramos los epac
Uruguay_Primaria_sin_epac <- filter(Uruguay_Primaria, NotasTercerT != "EPAC", NotasPrimerT != "EPAC", NotasSegundoT != "EPAC")

#Pasamos a INT los datos
Uruguay_Primaria_sin_epac$NotasTercerT <- as.numeric(Uruguay_Primaria_sin_epac$NotasTercerT)
Uruguay_Primaria_sin_epac$NotasSegundoT <- as.numeric(Uruguay_Primaria_sin_epac$NotasSegundoT)
Uruguay_Primaria_sin_epac$NotasPrimerT <- as.numeric(Uruguay_Primaria_sin_epac$NotasPrimerT)
#punto_part

In [ ]:
#wilcox
for (i in 1:6) {
  datos_filtrados <- subset(Uruguay_Primaria, asignatura == "MATEMATICA" & Año_cursado == i)

  #H0:La mediana de los promedios de Matematica del grado *i* es mayor o igual a 7.
  #H1:La mediana de los promedios de Matematica del grado *i* es menor que 7.
    aux <- wilcox.test(datos_filtrados$Promedio, mu = 7, alternative = "less")
  cat(i,"- Grado")
    print(aux)
}
#No hay informacion suficiente para rechazar h0.
#No hay evidencia suficiente para afirmar que el rendimiento en Matematica es bajo.

In [ ]:
#wilcox
for (i in 1:6) {
  datos_filtrados <- subset(Uruguay_Primaria, asignatura == "LENGUA" & Año_cursado == i)
  
  #H0:La mediana de los promedios de Lengua del grado *i* es mayor o igual a 7.
  #H1:La mediana de los promedios de Lengua del grado *i* es menor que 7.
  aux2 <- wilcox.test(datos_filtrados$Promedio, mu = 7, alternative = "less")
  cat(i,"- Grado")
    print(aux2)
}
#No hay informacion suficiente para rechazar h0.
#No hay evidencia suficiente para afirmar que el rendimiento en Lengua es bajo.

In [ ]:
#vemos si hay diferencia entre las notas del primer y tercer trimestre en general.
# hacemos el test
resultado_wilcox <- wilcox.test(Uruguay_Primaria_sin_epac$NotasTercerT, Uruguay_Primaria_sin_epac$NotasPrimerT, paired = TRUE)
print(resultado_wilcox)

if (resultado_wilcox$p.value < 0.05) {
  print("Hay una diferencia significativa entre las notas del primer y tercer trimestre.")
} else {
  print("No se encontró una diferencia significativa.")
}

In [ ]:
#Hacemos lo mismo pero ahora por cada grado con las notas de Matematica.
for (anio in 1:6) {
  datos_anio <- subset(Uruguay_Primaria_sin_epac, Año_cursado == anio & asignatura == "MATEMATICA")
    
    # hacemos el test
    resultado_wilcox <- wilcox.test(datos_anio$NotasTercerT, datos_anio$NotasPrimerT, paired = TRUE)
    cat("Grado:",anio)
    print(resultado_wilcox)
}
#En conclusion en todos los grados hay una diferencia significativa entre las notas del tercer y primer trimestre

In [ ]:
#vemos si hay diferencia entre las notas del primer y tercer trimestre con las notas de Lengua.
for (anio in 1:6) {
  datos_anio <- subset(Uruguay_Primaria_sin_epac, Año_cursado == anio & asignatura == "LENGUA")
    
    # hacemos el test
    resultado_wilcox <- wilcox.test(datos_anio$NotasTercerT, datos_anio$NotasPrimerT, paired = TRUE)
    cat("Grado:",anio)
    print(resultado_wilcox)
}

In [ ]:
for (anio in 1:6) {
  datos_anio <- subset(Uruguay_Primaria_sin_epac, Año_cursado == anio & asignatura == "MATEMATICA")
  
  # Crear un histograma para el grado actual
 hist(datos_anio$NotasTercerT, 
    col='#FC8D62',
    border='black', 
    breaks=c(6,7,8,9,10),
    xaxp=c(0,10,10),
    labels = TRUE,
    include.lowest=TRUE, 
    right=FALSE,
    plot=TRUE,
    density=100,
    xlab="Notas", 
    ylab="Frecuencia Absoluta", 
    main = paste("Comparación de Notas de Matematica 
    del primer y tercer trimestre -",anio, "Grado"),
    cex.main=1.1)

  hist(datos_anio$NotasPrimerT, 
        xaxp=c(0,10,10),
        breaks=c(6,7,8,9,10),
        border='black',
        labels = TRUE,
        include.lowest=TRUE,
        right=FALSE,
        add = TRUE, 
        col = '#FFD92F',
        density=100)

    # Agregar leyenda en el lado izquierdo
    legend("topleft", 
    legend = c("Primer Trimestre", "Tercer Trimestre"), 
    fill = c("#FFD92F", "#FC8D62"), 
    )

    mediana_tercer <- round(median(datos_anio$NotasTercerT),2)
    mediana_primer <- round(median(datos_anio$NotasPrimerT),2)
    media_tercer <- round(mean(datos_anio$NotasTercerT),2)
    media_primer <- round(mean(datos_anio$NotasPrimerT),2)
    
    # Agregar líneas verticales y etiquetas
    abline(v = mediana_tercer, col = '#FC8D62',lwd = 6)
    abline(v = mediana_primer, col = '#E78AC3',lwd = 6)
    
    abline(v = media_tercer, col = '#FF4040',lwd = 3, lty = 2)
    abline(v = media_primer, col = '#0000FF',lwd = 3, lty = 2)

    #leyenda de las medias
    legend("bottomleft", c("Media T1", "Media T3"),
       fill = c('#FF4040', '#0000FF'),
       xpd = TRUE,
       horiz = TRUE,
       bty = "n",
       inset = c(0, 0.75))

    #leyenda de las medianas
    legend("bottomleft", c("Mediana T1", "Mediana T3"),
           fill = c('#E78AC3', '#FC8D62'),
           xpd = TRUE,
           horiz = TRUE,
           bty = "n",
           inset = c(0, 0.80))
}

In [ ]:
for (anio in 1:6) {
  datos_anio <- subset(Uruguay_Primaria_sin_epac, Año_cursado == anio & asignatura == "LENGUA")
  
  # Crear un histograma para el año actual
 hist(datos_anio$NotasTercerT, 
    col='#FC8D62',
    border='black', 
    breaks=c(6,7,8,9,10),
    xaxp=c(0,10,10),
    labels = TRUE,
    include.lowest=TRUE, 
    right=FALSE,
    plot=TRUE,
    density=100,
    xlab="Notas", 
    ylab="Frecuencia Absoluta", 
    main = paste("Comparación de Notas de Lengua 
    del primer y tercer trimestre -",anio, "Grado"),
    cex.main=1.1)

  hist(datos_anio$NotasPrimerT, 
        xaxp=c(0,10,10),
        breaks=c(6,7,8,9,10),
        border='black',
        labels = TRUE,
        include.lowest=TRUE,
        right=FALSE,
        add = TRUE, 
        col = '#FFD92F',
        density=100)

    # Agregar leyenda en el lado izquierdo
    legend("topleft", 
    legend = c("Primer Trimestre", "Tercer Trimestre"), 
    fill = c("#FFD92F", "#FC8D62"), 
    )

    mediana_tercer <- round(median(datos_anio$NotasTercerT),2)
    mediana_primer <- round(median(datos_anio$NotasPrimerT),2)
    media_tercer <- round(mean(datos_anio$NotasTercerT),2)
    media_primer <- round(mean(datos_anio$NotasPrimerT),2)
    
    # Agregar líneas verticales y etiquetas
    abline(v = mediana_tercer, col = '#FC8D62',lwd = 6)
    abline(v = mediana_primer, col = '#E78AC3',lwd = 6)
    
    abline(v = media_tercer, col = '#FF4040',lwd = 3, lty = 2)
    abline(v = media_primer, col = '#0000FF',lwd = 3, lty = 2)

    #leyenda de las medias
    legend("bottomleft", c("Media T1", "Media T3"),
       fill = c('#FF4040', '#0000FF'),
       xpd = TRUE,
       horiz = TRUE,
       bty = "n",
       inset = c(0, 0.75))

    #leyenda de las medianas
    legend("bottomleft", c("Mediana T1", "Mediana T3"),
           fill = c('#E78AC3', '#FC8D62'),
           xpd = TRUE,
           horiz = TRUE,
           bty = "n",
           inset = c(0, 0.80))
}

In [ ]:
#Verificamos si los promedios de los alumnos de lengua y Matematica siguen una distribucion normal
primaria_matematica_lengua <- subset(Uruguay_Primaria, asignatura %in% c("MATEMATICA", "LENGUA"))
primaria_matematica_lengua <- na.omit(primaria_matematica_lengua)

for (x in 1:6){
    matematica <- filter(primaria_matematica_lengua, asignatura == "MATEMATICA" & Año_cursado==x)
    lengua<- filter(primaria_matematica_lengua, asignatura =="LENGUA" & Año_cursado==x)

    suppressWarnings({
    cat("TEST de normalidad, promedios MATEMATICA",x,"Grado\n")
    aux<-ks.test(matematica$Promedio,"pnorm")
    print(aux)

    cat("TEST de normalidad, promedios LENGUA",x,"Grado\n")
    aux2<-ks.test(lengua$Promedio,"pnorm")
    print(aux2)
        })
    }
#llegamos a la conclusion de que ninguno sigue una distribucion normal

In [ ]:
# HIPOTESIS TEST LEVENE 
# Hipótesis nula (H0): Las varianzas de las notas de los estudiantes en Matemática y Lengua son iguales.
# Hipótesis alternativa (H1): Las varianzas de las notas de los estudiantes en Matemática y Lengua no son iguales.

# Test de Wilcoxon-Mann-Whitney para Matematica y Lengua
#H0: Las distribuciones de las notas de los estudiantes en Matemática y Lengua son iguales.
#H1: Las distribuciones de las notas de los estudiantes en Matemática y Lengua son diferentes.

var_test_results <- list()
wilcox_test_results <- list()

# Realizar los tests para cada grado desde primero hasta sexto
for (i in 1:6) {
  datos_año <- subset(primaria_matematica_lengua, Año_cursado == i)
  datos_año <- datos_año %>% distinct(Documento, .keep_all = TRUE)
  
  # Realizar el test de Levene para la comparación de varianzas
  var_test_result <- var.test(datos_año$Promedio[datos_año$asignatura == "MATEMATICA"], datos_año$Promedio[datos_año$asignatura == "LENGUA"])
  var_test_results[[i]] <- var_test_result
  
  # Realizar el test de Wilcoxon-Mann-Whitney para Matemática y Lengua
  wilcox_test_result <- wilcox.test(datos_año$Promedio[datos_año$asignatura == "MATEMATICA"], datos_año$Promedio[datos_año$asignatura == "LENGUA"])
  wilcox_test_results[[i]] <- wilcox_test_result # lo guardamos en una lista
}

# Imprimir los resultados de los tests
for (i in 1:6) {
  cat("Grado:", i, "\n")
  cat("Resultado del test de Levene:\n")
  print(var_test_results[[i]])
  cat("Resultado del test de Wilcoxon-Mann-Whitney:\n")
  print(wilcox_test_results[[i]])
}

In [ ]:
#plot(Primaria_por_dep$Primero)

ggplot(Primaria_por_dep, aes(x = Sector, y = Todos, fill = X.c1.mbito)) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(title = "Cantidad de matriculados por sector y ámbito") +
  xlab("Sector") + ylab("Cantidad de matriculados") +
  theme_minimal()

#test de normalidad
ks.test(Primaria_por_dep$Todos, "pnorm")
ks.test(Primaria_por_dep$Primero, "pnorm")
ks.test(Primaria_por_dep$Segundo, "pnorm")
ks.test(Primaria_por_dep$Tercero, "pnorm")
ks.test(Primaria_por_dep$Cuarto, "pnorm")
ks.test(Primaria_por_dep$Quinto, "pnorm")
ks.test(Primaria_por_dep$Sexto, "pnorm")

In [ ]:
datos <- gather(Primaria_por_dep, key = "Ciclo", value = "Conteo", Primero:Sexto)

datos <- datos %>% select(-Total.Segundo.Ciclo, -Todos, -Departamento, -X, -Sector, -Total.Primer.Ciclo, -X.c1.mbito)

datos$Ciclo <- as.factor(datos$Ciclo)
datos$Conteo <- as.numeric(datos$Conteo)
datos
#los datos no siguen una distribucion normal, no tomar en cuenta este test, es solo de visualizacion.
modelo_anova <- aov(Conteo ~ Ciclo, data = datos)
summary(modelo_anova)

# Realizar el test post-hoc de Tukey
TukeyHSD(modelo_anova)

# Graficar los residuos
qqnorm(modelo_anova$residuals)
qqline(modelo_anova$residuals)

# Realizar el test de Shapiro-Wilk para normalidad de los residuos
shapiro.test(modelo_anova$residuals)

# Homogeneidad de las Varianzas con el test de Levene
leveneTest(Conteo ~ Ciclo, data = datos)

# Calcular la potencia del ANOVA
potencia <- pwr.anova.test(k = 6, n = 1731, f = 0.2, sig.level = 0.05)
potencia

In [ ]:
# Transformar datos a formato largo
datos <- gather(Primaria_por_dep, key = "Ciclo", value = "Conteo", Primero:Sexto)

datos <- datos %>% select(-Total.Segundo.Ciclo, -Todos, -Departamento, -X, -Sector, -Total.Primer.Ciclo, -Ciclo)

datos$X.c1.mbito <- as.factor(datos$X.c1.mbito)
datos$Conteo <- as.numeric(datos$Conteo)

datos
#H0: Las medias de los conteos de estudiantes en los diferentes ambitos son iguales.
#H1: Las medias de los conteos de estudiantes en los dif ambitos son distintos.

print("test de anova (no tomar en cuenta)")
resultado_anova <- aov(Conteo ~ X.c1.mbito, data = datos)
summary(resultado_anova)
TukeyHSD(resultado_anova)
#aunque no son datos normales, es solo para ver

#TEST CORRESPONDIENTE
cat("\n","TEST de Kruskal")
resultado_kruskal <- kruskal.test(Conteo ~ X.c1.mbito, data = datos) # p-value = 3.15e-06 | RECHAZAMOS H0
print(resultado_kruskal)

In [ ]:
#Realizamos un test F (ANOVA). Sin embargo es solo para visualizar ya que los datos no cumplen con los requisitos para aplicar este test

filter_mate <- filter(Uruguay_Primaria_sin_epac, asignatura =="MATEMATICA")

# H0: Las distribuciones de los promedios de Matematica son iguales para los diferentes turnos.
# H1: Al menos una de las distribuciones de los promedios de Matematica es diferente entre los turnos.
print("Solo visualizacion, no tomar en cuenta")
resultado_anova <- aov(Promedio ~ Turno, data = filter_mate)
summary(resultado_anova)

#Realizamos el test correspondiente para estos datos.
kruskal_matematica <- kruskal.test(Promedio ~ Turno, data = filter_mate)
print(kruskal_matematica)

In [ ]:
#Realizamos un test F (ANOVA). Sin embargo es solo para visualizar ya que los datos no cumplen con los requisitos para aplicar este test
for (x in 1:6){
filter_mate <- filter(Uruguay_Primaria_sin_epac, asignatura =="MATEMATICA", Año_cursado== x)

   #hipotesis para anova
#H0: Las medias de los promedios de Matemática son iguales para los diferentes turnos.
#H1: Al menos una de las medias de los promedios de Matemática es diferente entre los turnos.
print("Solo visualizacion, no tomar en cuenta")
    cat("Grado:", x, "\n")
resultado_anova <- aov(Promedio ~ Turno, data = filter_mate)
aux<-summary(resultado_anova)
    print(aux)

  #hipotesis para kruskal
#H0: Las distribuciones de los promedios de Matemática son iguales para los diferentes turnos
#H1: Al menos una de las distribuciones de los promedios de Matemática es diferente entre los turnos
 # Test de Kruskal-Wallis (test correspondiente)
cat("\nTest de Kruskal-Wallis (test correspondiente)\n")
kruskal_matematica <- kruskal.test(Promedio ~ Turno, data = filter_mate)
print(kruskal_matematica)
    }

In [ ]:
#Realizamos un test F (ANOVA). Sin embargo es solo para visualizar ya que los datos no cumplen con los requisitos para aplicar este test
for (x in 1:6){
filter_leng <- filter(Uruguay_Primaria_sin_epac, asignatura =="LENGUA", Año_cursado== x)

   #hipotesis para anova
#H0: Las medias de los promedios de Lengua son iguales para los diferentes turnos.
#H1: Al menos una de las medias de los promedios de Lengua es diferente entre los turnos.
print("Solo visualizacion, no tomar en cuenta")
    cat("Grado:", x, "\n")
resultado_anova <- aov(Promedio ~ Turno, data = filter_leng)
aux<-summary(resultado_anova)
    print(aux)

  #hipotesis para kruskal
#H0: Las distribuciones de los promedios de Lengua son iguales para los diferentes turnos
#H1: Al menos una de las distribuciones de los promedios de Lengua es diferente entre los turnos
 # Test de Kruskal-Wallis (test correspondiente)
cat("\nTest de Kruskal-Wallis (test correspondiente)\n")
kruskal_matematica <- kruskal.test(Promedio ~ Turno, data = filter_leng)
print(kruskal_matematica)
    }

In [ ]:
# Filtrar el conjunto de datos para el sexto grado y las asignaturas especificadas
sexto_anio <- filter(Uruguay_Primaria, (asignatura == "LENGUA" | asignatura== "MATEMATICA" 
                                         | asignatura =="CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES") & Año_cursado== 6)

# guardamos los dni de los estudiantes que realizaron las 4 materias
documentos_completos <- sexto_anio %>%
  group_by(Documento) %>%
  filter(n_distinct(asignatura) == 4) %>%
  distinct(Documento)

# Filtrar el dataset original para obtener solo los registros de los alumnos que realizan las 4 asignaturas
sexto_anio_completo <- Uruguay_Primaria %>% filter(Documento %in% documentos_completos$Documento)

aux <- filter(sexto_anio_completo, (asignatura == "LENGUA" | asignatura== "MATEMATICA" 
                                         | asignatura =="CIENCIAS SOCIALES" | asignatura == "CIENCIAS NATURALES") & Año_cursado== 6)
head(aux)

#ordenamos los datos
promedios_alumnos <- aux %>%
  select(Documento, asignatura, Promedio) %>%
  spread(key = asignatura, value = Promedio)

# Mostrar el dataset con los promedios
head(promedios_alumnos,15)

#eliminamos las filas que tienen valores faltantes:
promedios_alumnos_clean <- promedios_alumnos %>%
  drop_na()
head(promedios_alumnos_clean,15)

In [ ]:
#cambiamos los nombres de las variables que tienen espacios
names(promedios_alumnos_clean) <- make.names(names(promedios_alumnos_clean)) 
head(promedios_alumnos_clean,2)

In [ ]:
#No tomar en cuenta, los datos no cumplen con los requisitos requeridos. Solo es para visualizacion.
# Creación de datos
set.seed(123)

# Realización del ANCOVA
modelo <- lm(CIENCIAS.SOCIALES ~ CIENCIAS.NATURALES + LENGUA + MATEMATICA, data = promedios_alumnos_clean)
summary(modelo)

# Residuos del modelo
residuos <- residuals(modelo)

# Prueba de Shapiro-Wilk para la normalidad de los residuos
shapiro.test(residuos)

# Calcular los VIF para cada covariable en el modelo
vif(modelo)
#Si alguno de los VIF es mayor que 10, indicaría la presencia de colinealidad entre las covariables correspondientes en el modelo. 

# Gráfico de dispersión de los residuos vs. cada covariable 

#Idealmente, los residuos deberían distribuirse alrededor de cero de manera aleatoria,
#sin mostrar un patrón evidente en función de ninguna covariable. 
#Si los residuos muestran un patrón sistemático, podría indicar una violación del supuesto de homogeneidad de las varianzas.

plot(residuos ~ CIENCIAS.NATURALES, data = promedios_alumnos_clean, xlab = "CIENCIAS NATURALES", ylab = "Residuos", main = "Residuos vs CIENCIAS NATURALES")
abline(h = 0, col = "red")

plot(residuos ~ LENGUA, data = promedios_alumnos_clean, xlab = "LENGUA", ylab = "Residuos", main = "Residuos vs LENGUA")
abline(h = 0, col = "blue")

plot(residuos ~ MATEMATICA, data = promedios_alumnos_clean, xlab = "MATEMATICA", ylab = "Residuos", main = "Residuos vs MATEMATICA")
abline(h = 0, col = "green")

In [ ]:
#No tomar en cuenta, los datos no cumplen con los requisitos requeridos. Solo es para visualizacion.
Uruguay_Primaria$Cantidad_enpro <- ave(Uruguay_Primaria$Resultado == "En proceso", Uruguay_Primaria$Documento, FUN = sum)

# Filtrar para el primer grado de Matemática
datos_primer_anio_matematica <- filter(Uruguay_Primaria, asignatura == "MATEMATICA" & Año_cursado == 1)
#head(datos_primer_anio_matematica)

# test de MANOVA, no tomar en cuenta, solo visualizacion. (Los datos no cumplen los requisitos requeridos)
modelo_manova <- manova(cbind(Cantidad_enpro, Promedio) ~ NotasTercerT, data = datos_primer_anio_matematica)
summary(modelo_manova)

mardia2<-function(datos_primer_anio_matematica){
  n<-nrow(datos_primer_anio_matematica)
  k<-ncol(datos_primer_anio_matematica)

  x<-as.matrix(datos_primer_anio_matematica)
  media<-colMeans(datos_primer_anio_matematica)
  matriz.media<-matrix(media,nrow=n,ncol=k,byrow=TRUE)

  S<-cov(datos_primer_anio_matematica)*(n-1)/n
  SI<-solve(S)   #Inversa de la matriz de covarianzas

  distm<-((x-matriz.media)%*%SI)%*%t((x-matriz.media))

  gl<-k*(k+1)*(k+2)/6

  # Medida de Asimetría Multivariada
  AM<-sum(distm^3)/n^2

  if (n<30){
  # Test Multivariado de Asimetría de Mardia - Muestras Pequeñas
  c<-(n+1)*(n+3)*(k+1)/(n*((n+1)*(k+1)-6))
  TAM<-n*c/6*AM
  valor_p1<-pchisq(TAM,gl,lower.tail = FALSE)
  } else {
    # Test Multivariado de Asimetríıa de Mardia - Muestras grandes
  TAM<-n/6*AM
  valor_p1<-pchisq(TAM,gl,lower.tail = FALSE)
  }
  #Medida de Kurtosis Multivariado
  KM<-sum((diag(distm))^2)/n

  #Test Multivariado de Kurtosis de Mardia
  TKM<-(KM-k*(k+2))*sqrt(n/(8*k*(k+2)))
  TKM2<-TKM^2
  valor_p2<-pchisq(TKM2,1,lower.tail = FALSE)

  cat("Test de Normalidad Multivariada", "\n" ,
      "Ho: Los datos siguen una Distribución Normal Multivariada","\n")
  cat("----------------------------------------------------------------","\n")
  cat("Prueba de Mardia - Asimetría","\n",
      "Asimetría Multivariada =", AM, "\n",
      "Valor de TAM =",TAM, "\n",
      "Valor p=", valor_p1, "\n")
  cat("----------------------------------------------------------------","\n")
  cat("Prueba de Mardia - Curtosis","\n",
      "Curtosis Multivariado =", KM, "\n",
      "Valor de TKM =",TKM2, "\n",
      "Valor p=", valor_p2, "\n")
     }

residuos <- residuals(modelo_manova)
mardia2(residuos)

# Prueba de homogeneidad de las varianzas-covarianzas
mauchly_test <- mauchly.test(modelo_manova)
mauchly_test

In [ ]:
#ANOVA DE DOS FACTORES REPETIDOS

#H0: Las medias de las notas de Matematica en los tres trimestres de los alumnos de primer grado del turno completo son iguales.
#H1: Al menos una de las medias de las notas de Matematica en los tres trimestres de los alumnos de primer grado del turno completo es diferente.

primaria_filtrado <- Uruguay_Primaria_sin_epac %>%
  filter(Año_cursado == 1 & asignatura == "MATEMATICA" & Turno == "COMPLETO")
# Seleccionar y convertir las columnas necesarias
primaria_DOS_FACTORES <- primaria_filtrado %>%
  dplyr::select(Documento, NotasPrimerT, NotasSegundoT, NotasTercerT) %>%
  mutate(Documento = as.integer(Documento),
         NotasPrimerT = as.double(NotasPrimerT),
         NotasSegundoT = as.double(NotasSegundoT),
         NotasTercerT = as.double(NotasTercerT)) %>%
  distinct()

# Verificar los primeros registros para confirmar los cambios
primaria_DOS_FACTORES

# Convertir los datos a formato largo
datos_trimestres <- melt(primaria_DOS_FACTORES, id.vars = "Documento", variable.name = "Trimestres", value.name = "Notas")

# Realizar el ANOVA de medidas repetidas

modelo_anova <- aov(Notas ~ Trimestres + Error(Documento/Trimestres), data = datos_trimestres )

# Cambiar el tipo de modelo a lm
modelo_lm <- lm(modelo_anova)

# Calcular Anova con el modelo lm
anova_result <- Anova(modelo_lm, type = "III")
print(anova_result)

#Supuestos

# Prueba de normalidad de los residuos
shapiro.test(residuals(modelo_lm))

# Prueba de homogeneidad de varianzas (Levene's Test)
#leveneTest(residuals(modelo_lm) ~ datos_trimestres$Trimestres)

# Prueba de homogeneidad de varianzas usando var.test por pares
var_test_result_1_2 <- var.test(Notas ~ Trimestres, data = subset(datos_trimestres, Trimestres %in% c("NotasPrimerT", "NotasSegundoT")))
var_test_result_1_3 <- var.test(Notas ~ Trimestres, data = subset(datos_trimestres, Trimestres %in% c("NotasPrimerT", "NotasTercerT")))
var_test_result_2_3 <- var.test(Notas ~ Trimestres, data = subset(datos_trimestres, Trimestres %in% c("NotasSegundoT", "NotasTercerT")))

print(var_test_result_1_2)
print(var_test_result_1_3)
print(var_test_result_2_3)
# Prueba de linealidad (no se proporciona un test específico en R, pero se puede verificar visualmente)
plot(modelo_lm, which = 1)


# Calcular la potencia del ANOVA
potencia_anova <- pwr.anova.test(k = 3, n = 20, sig.level = 0.05, f = 0.25)$power

print("potencia_anova")
potencia_anova


In [ ]:
#ANOVA DE DOS FACTORES REPETIDOS

#H0: Las medias de las notas de Matematica en los tres trimestres de los alumnos de segundo grado del turno completo son iguales.
#H1: Al menos una de las medias de las notas de Matematica en los tres trimestres de los alumnos de segundo grado del turno completo es diferente.

primaria_filtrado <- Uruguay_Primaria_sin_epac %>%
  filter(Año_cursado == 2 & asignatura == "MATEMATICA" & Turno == "COMPLETO")
# Seleccionar y convertir las columnas necesarias
primaria_DOS_FACTORES <- primaria_filtrado %>%
  dplyr::select(Documento, NotasPrimerT, NotasSegundoT, NotasTercerT) %>%
  mutate(Documento = as.integer(Documento),
         NotasPrimerT = as.double(NotasPrimerT),
         NotasSegundoT = as.double(NotasSegundoT),
         NotasTercerT = as.double(NotasTercerT)) %>%
  distinct()

# Verificar los primeros registros para confirmar los cambios
primaria_DOS_FACTORES

# Convertir los datos a formato largo
datos_trimestres <- melt(primaria_DOS_FACTORES, id.vars = "Documento", variable.name = "Trimestres", value.name = "Notas")

# Realizar el ANOVA de medidas repetidas

modelo_anova <- aov(Notas ~ Trimestres + Error(Documento/Trimestres), data = datos_trimestres )

# Cambiar el tipo de modelo a lm
modelo_lm <- lm(modelo_anova)

# Calcular Anova con el modelo lm
anova_result <- Anova(modelo_lm, type = "III")
print(anova_result)

#Supuestos

# Prueba de normalidad de los residuos
shapiro.test(residuals(modelo_lm))

# Prueba de homogeneidad de varianzas (Levene's Test)
#leveneTest(residuals(modelo_lm) ~ datos_trimestres$Trimestres)

# Prueba de homogeneidad de varianzas usando var.test por pares
var_test_result_1_2 <- var.test(Notas ~ Trimestres, data = subset(datos_trimestres, Trimestres %in% c("NotasPrimerT", "NotasSegundoT")))
var_test_result_1_3 <- var.test(Notas ~ Trimestres, data = subset(datos_trimestres, Trimestres %in% c("NotasPrimerT", "NotasTercerT")))
var_test_result_2_3 <- var.test(Notas ~ Trimestres, data = subset(datos_trimestres, Trimestres %in% c("NotasSegundoT", "NotasTercerT")))

print(var_test_result_1_2)
print(var_test_result_1_3)
print(var_test_result_2_3)
# Prueba de linealidad (no se proporciona un test específico en R, pero se puede verificar visualmente)
plot(modelo_lm, which = 1)


# Calcular la potencia del ANOVA
potencia_anova <- pwr.anova.test(k = 3, n = 20, sig.level = 0.05, f = 0.25)$power

print("potencia_anova")
potencia_anova

In [ ]:
#H0: Las medianas de las notas de los estudiantes en escuelas privadas y públicas son iguales.
#H1: Las medianas de las notas de los estudiantes en escuelas privadas y públicas son diferentes.

# Filtrar los datos para obtener solo los estudiantes únicos por Documento
datos_unicos <- Uruguay_Primaria_sin_epac %>% distinct(Documento, .keep_all = TRUE)

# Lista para almacenar los resultados de los tests
ks_test_results <- list()
var_test_results <- list()
wilcox_test_results <- list()

for (i in 1:6) {
  datos_anio <- subset(datos_unicos, Año_cursado == i)
  
  # Separar los datos por si son de escuela privada o pública
  datos_privada <- subset(datos_anio, EsPrivada == "S")
  datos_publica <- subset(datos_anio, EsPrivada == "N")
  
  #Test de Kolmogorov-Smirnov
  ks_test_result <- suppressWarnings(ks.test(datos_privada$Promedio, datos_publica$Promedio))
  
  #Test de varianza
  var_test_result <- var.test(datos_privada$Promedio, datos_publica$Promedio)
  
  #Test de Wilcoxon-Mann-Whitney
  wilcox_test_result <- wilcox.test(datos_privada$Promedio, datos_publica$Promedio)
  
  # Guardar los resultados en las listas
  ks_test_results[[i]] <- ks_test_result
  var_test_results[[i]] <- var_test_result
  wilcox_test_results[[i]] <- wilcox_test_result
}

# Mostrar los resultados
for (i in 1:6) {
  cat("Año:", i, "\n")
  cat("Resultado del test de Kolmogorov-Smirnov:\n")
  print(ks_test_results[[i]])
  cat("\n")
  cat("Resultado del test de varianza:\n")
  print(var_test_results[[i]])
  cat("\n")
  cat("Resultado del test de Wilcoxon-Mann-Whitney:\n")
  print(wilcox_test_results[[i]])
  cat("\n")
}

In [ ]:
#TEST DE FRIEDMAN
#H0: No hay diferencias significativas en las medianas de los promedios de las tres materias para el turno mañana en primer grado
#H1:Al menos una de las medianas de los promedios de las tres materias es significativamente diferente para el turno mañana en primer grado

alumnos_mana <- Uruguay_Primaria_sin_epac %>%
  filter(Turno == "MAÑANA") %>%
  dplyr::select(Documento) %>%
  distinct()

primer_ano <- Uruguay_Primaria_sin_epac %>%
  filter(Año_cursado == 1 & asignatura %in% c("MATEMATICA", "LENGUA", "CIENCIAS SOCIALES") & Turno == "MAÑANA")

# Identificar los documentos de los alumnos que realizan las 3 asignaturas
documentos_completos <- primer_ano %>%
  group_by(Documento) %>%
  filter(n_distinct(asignatura) == 3) %>%
  distinct(Documento)

# Filtrar el dataset original para obtener solo los registros de los alumnos que realizan las 3 asignaturas en la modalidad Agrotécnica
primer_ano_completo <- Uruguay_Primaria_sin_epac %>%
  filter(Año_cursado == 1 & Documento %in% documentos_completos$Documento & asignatura %in% c("MATEMATICA", "LENGUA", "CIENCIAS SOCIALES") & Turno == "MAÑANA")

# Seleccionar las columnas relevantes y reorganizar los datos al formato ancho
promedios_alumnos <- primer_ano_completo %>%
  dplyr::select(Documento, asignatura, Promedio) %>%
  pivot_wider(names_from = asignatura, values_from = Promedio)

promedios_alumnos <- promedios_alumnos %>% rename_with(make.names)

promedios_alumnos

In [ ]:
#test de friedman
# Extraer los vectores para cada materia
CIENCIAS.SOCIALES <- promedios_alumnos$CIENCIAS.SOCIALES
LENGUA <- promedios_alumnos$LENGUA
MATEMATICA <- promedios_alumnos$MATEMATICA

# Crear los vectores concatenados de promedios y materias
promedios <- as.numeric(c(CIENCIAS.SOCIALES, LENGUA, MATEMATICA))
materias <- as.factor(c(rep("CIENCIAS.SOCIALES", length(CIENCIAS.SOCIALES)), 
                        rep("LENGUA", length(LENGUA)), 
                        rep("MATEMATICA", length(MATEMATICA))))

# Verificar la longitud de los vectores
length(promedios)
length(materias)

# Asegurar que el vector 'rendimiento' tenga la longitud correcta3
alumno <- rep(1:length(CIENCIAS.SOCIALES), 3)

# Crear el data frame
datos <- data.frame(alumno, promedios, materias)

# Mostrar los primeros registros del data frame
datos

by(data = datos$promedios, INDICES = datos$materias, FUN = median)

friedman.test(promedios ~ materias| alumno, data=datos)

n <- nrow(datos) / 3
# Realizar un análisis de varianza (ANOVA) en rangos
friedman_anova <- pwr.anova.test(k = 3, # Número de grupos
                                 n = n, 
                                 f = 0.25, # Tamaño del efecto F
                                 sig.level = 0.05) # Nivel de significancia

# Extraer la potencia del resultado del análisis de varianza
potencia <- friedman_anova$power

potencia

In [ ]:
conteo_materias <- table(datos$materias)
conteo_materias

In [ ]:
#TEST DE COCHRAN 
#H0: Las proporciones de las notas en Matemática de los alumnos de primer grado de la escuela de al menos 8 estudiantes son iguales en todos los trimestres.
#H1: Por lo menos una de las proporciones de las notas en matemática de los alumnos de primer grado de la escuela de al menos 8 estudiantes es diferente en alguno de los trimestres

primer_grado <- filter(Uruguay_Primaria, Año_cursado== 1)

# Identificar una escuela que tenga al menos 10 estudiantes de Matematica
escuela_min_10_estudiantes <- primer_grado %>%
  filter(asignatura == "MATEMATICA") %>%
  count(CUE) %>%
  filter(n >= 10) %>%
  slice(1)  # agarramos la primera fila que cumple con la condicion

# Filtrar el dataset principal
Primaria_filtro <- primer_grado %>%
  filter(CUE %in% escuela_min_10_estudiantes$CUE,      
         asignatura == "MATEMATICA") %>%  # Solo estudiantes que cursan Matematica
  mutate(Resultado = ifelse(Resultado == "Aprobado", 1, 
                            ifelse(Resultado == "En proceso", 0, Resultado))) #pasamos a dicotomico 1 o 0

head(Primaria_filtro)

# Contar cuántos documentos de alumnos cumplen con el criterio
num_documentos <- nrow(Primaria_filtro)

# Mostrar el resultado
print(paste("Número de documentos de alumnos que cumplen con el criterio:", num_documentos))

# Filtrar las columnas Documento, NotasPrimerT, NotasSegundoT, NotasTercerT y Resultado, y eliminar duplicados
Primaria_filtrado_coch <- Primaria_filtro %>%
  dplyr::select(Documento, NotasPrimerT, NotasSegundoT, NotasTercerT, Resultado) %>%
  distinct() %>%
  mutate(Documento = as.integer(Documento),
    Resultado = as.double(Resultado))

Primaria_filtrado_coch <- Primaria_filtrado_coch %>%
  mutate(
    NotasPrimerT = case_when(
      NotasPrimerT == "EPAC" ~ 0,
      NotasPrimerT >= 6 ~ 1,
      NotasPrimerT == "10" ~ 1),
    NotasSegundoT = case_when(
      NotasSegundoT == "EPAC" ~ 0,
      NotasSegundoT >= 6 ~ 1,
      NotasSegundoT == "10" ~ 1),
    NotasTercerT = case_when(
      NotasTercerT == "EPAC" ~ 0,
      NotasTercerT >= 6 ~ 1,
      NotasTercerT == "10" ~ 1)
  )

# Verificar los primeros registros para confirmar los cambios
Primaria_filtrado_coch

In [ ]:
T1 <- Primaria_filtrado_coch$NotasPrimerT
T2 <- Primaria_filtrado_coch$NotasSegundoT
T3 <- Primaria_filtrado_coch$NotasTercerT

trimestres= as.numeric(c(T1,T2,T3))
resultado <- as.factor(c(rep("T1",length(T1)), rep("T2",length(T2)),rep("T3",length(T3))))

# Crear el marco de datos para el test de Cochran
PUNTAJES <- data.frame(
  escuelas = rep(1:length(T1), 3),
  resultado = resultado,
  trimestres = trimestres
)

PUNTAJES
# Realizar el test de Cochran
resultado_test <- CochranQTest(trimestres ~ resultado | escuelas, data = PUNTAJES)
resultado_test

# Definir los parámetros del test
k <- 3  # Número de grupos
alpha <- 0.05  # Nivel de significancia

# Obtener el valor crítico para el nivel de significancia dado
q_alpha <- qchisq(1 - alpha, df = k - 1)

# Calcular la potencia
potencia <- 1 - pchisq(q_alpha, df = k - 1, ncp = 0, lower.tail = FALSE)

potencia

In [ ]:
#Armo los condicionales para hacer la columna desempenio

Uruguay_Primaria <- Uruguay_Primaria %>%
  mutate(Desempenio = case_when(
    Promedio < 7 ~ "aprobado",
    Promedio >= 7 & Promedio < 8 ~ "bueno",
    Promedio >= 8 & Promedio < 9 ~ "muy bueno",
    Promedio >= 9 ~ "distinguido",
    is.na(Promedio) & NotasPrimerT == "EPAC" & NotasSegundoT == "EPAC" & NotasTercerT == "EPAC" ~ "insuficiente",
    is.na(Promedio) & NotasTercerT == "EPAC" ~ "regular",
    is.na(Promedio) & rowSums(cbind(NotasPrimerT, NotasSegundoT, NotasTercerT) == "EPAC") == 2 ~ "regular",
    is.na(Promedio) & rowSums(cbind(NotasPrimerT, NotasSegundoT, NotasTercerT) == "EPAC") == 1 & NotasTercerT != "EPAC" ~ "aprobado",
  ))

#ordenamos los valores de desempenio
Uruguay_Primaria$Desempenio <- factor(Uruguay_Primaria$Desempenio, 
                                    levels = c("insuficiente", "regular", "aprobado", "bueno", "muy bueno", "distinguido"))

In [ ]:
table(Uruguay_Primaria$Desempenio)

Tablas de contingencia por turnos en Lengua de primer grado

In [ ]:
primero_lengua <- filter(Uruguay_Primaria, Año_cursado== 1, asignatura== "LENGUA")
primero_lengua <- subset(primero_lengua, Turno != "ROTATIVO" & Turno != "COMPLETO") #sacamos los turnos rotativo y completo
                                                                                    #porque no cumple con tener minimo 5 "muestras" en cada celda
tabla_contingencia_primero <- table(primero_lengua$Turno, primero_lengua$Desempenio)

tabla_contingencia_primero

In [ ]:
#Tuve que cambiar un poco las tablas por que antes no daban bien los numeros de Total
#porcentaje total
cat("\n","Porcentaje Total","\n")
porcentaje_total <- round(prop.table(tabla_contingencia_primero) * 100,1)
porcentaje_total<-cbind(porcentaje_total,"Total"=margin.table(porcentaje_total, 1))
porcentaje_total<-rbind(porcentaje_total,"Total"=margin.table(porcentaje_total, 2))
print(porcentaje_total)

#porcentaje por fila
cat("\n","Porcentaje Fila","\n")
porcentaje_fila<-round(prop.table(rbind(tabla_contingencia_primero,"Total"=margin.table(tabla_contingencia_primero, 2)),1)*100,1)
porcentaje_fila<-cbind(porcentaje_fila,"Total"=margin.table(porcentaje_fila, 1))
print(porcentaje_fila)

#porcentaje por columna
cat("\n","Porcentaje Columna", "\n")
porcentaje_columna<-round(prop.table(cbind(tabla_contingencia_primero,"Total"=margin.table(tabla_contingencia_primero, 1)),2)*100,1)
porcentaje_columna<-rbind(porcentaje_columna,"Total"=margin.table(porcentaje_columna, 2))
print(porcentaje_columna)

In [ ]:
porcentaje_fila <- porcentaje_fila[rownames(porcentaje_fila) != "Total", ]
porcentaje_fila <- porcentaje_fila[, colnames(porcentaje_fila) != "Total"]

porcentaje_columna <- porcentaje_columna[rownames(porcentaje_columna) != "Total", ]
porcentaje_columna <- porcentaje_columna[, colnames(porcentaje_columna) != "Total"]

In [ ]:
#test de chi-cuadrado
cat("\n","TEST de chi cuadrado")
resultado_chi <- chisq.test(tabla_contingencia_primero)
resultado_chi

#test de asosiacion cramerV
cat("\n","test de cramerV")
CramerV(tabla_contingencia_primero)

#tabla de valores esperados
cat("\n","Valores esperados")
round(resultado_chi$expected,0)

#residuos
cat("\n","Residuos")
round(resultado_chi$residuals, 1)

In [ ]:
aux <- t(porcentaje_fila)

In [ ]:
bpc<-barplot(aux,
        main = "Gráfico de Contigencia por Fila",
        beside = TRUE, # Barras agrupadas
        space = c(0.1, 2.5), # Espacio primer elemento es el espacio entre barras de cada grupo (0.1) y el segundo el espacio entre grupos (2.5).
        ylim = c(0,100),
        cex.axis = 1.2, #tamaño etiquetas eje y
        cex.names=1.5, #tamaño etiqueta eje x
        cex.main=1.5, #tamaño título gráfico
        col=c("darkblue","green", "yellow","#364972","#6E1772","#1E7230"),
        xlab = "Turno",
        cex.lab=1.5, #tamaño  xlab ylab
        ylab = "Porcentaje",
       col.lab = "blue") # Color xlab ylab

#leyenda
legend("topright", inset = c(0, 0), # Ajusta la posición de la leyenda
       legend = rownames(aux),
       title = "Desempeño", # Título
       title.adj = 0.5,          # Ajuste horizontal del título
       title.col = "black",       # Color del título
       pch = 15,  # Cambia a cuadrados
       col = c("darkblue", "green", "yellow", "#364972", "#6E1772", "#1E7230"),
       pt.cex = 2,  # Tamaño de los cuadrados
       cex = 1.32)  # Cambiar el tamaño del texto

# valores sobre barras
TC <- aux
text(bpc, TC+2.5,format(TC), cex=1.2)


In [ ]:
df <- as.data.frame(as.table(porcentaje_columna))
colnames(df) <- c("Var1", "Var2", "Freq")

ggplot(data=df,aes(x=Var2,y=Freq,fill=Var1))+
  geom_bar(stat="identity",colour="blue")+
    scale_fill_manual(name= "Turnos", values=c("#5F9EA0","#1C7202"))+
   geom_text(aes(label=Freq),position=position_stack(vjust=0.41),size = 4, color = "red")+
  xlab("Desempeño")+
  ylab("Porcentaje")+
 ggtitle("Gráfico de Contigencia por columna") +
 theme(axis.title = element_text(size = 15, color = "black", face = "bold"))+
 theme(title = element_text(size = 15, color = "black", face = "bold"))+
  theme(plot.title = element_text(hjust = 0.5)) +#centra titulo
 theme(legend.text = element_text(size = 12, color = "black", face= "bold"))+
 theme(axis.text = element_text(size = 10, color = "black", face= "bold"))


In [ ]:
columna2 <- t(porcentaje_columna)
bpc <- barplot(columna2,
               main = "Gráfico de Contigencia por Columna",
               beside = TRUE, # Barras agrupadas
               space = c(0.1, 1.5), # Ajuste del espacio
               ylim = c(0, 100),
               cex.axis = 1.2, # Tamaño etiquetas eje y
               cex.names = 1.5, # Tamaño etiqueta eje x
               cex.main = 1.5, # Tamaño título gráfico
               col = c("darkblue", "green", "yellow", "#364972", "#6E1772", "#1E7230"),
               xlab = "Turno",
               cex.lab = 1.5, # Tamaño xlab ylab
               ylab = "Porcentaje",
               col.lab = "blue") # Color xlab ylab

# Leyenda
legend("topright", inset = c(0, 0), # Ajusta la posición de la leyenda
       legend = rownames(columna2),
       title = "Desempeño", # Título
       title.adj = 0.5, # Ajuste horizontal del título
       title.col = "black", # Color del título
       pch = 15, # Cambia a cuadrados
       col = c("darkblue", "green", "yellow", "#364972", "#6E1772", "#1E7230"),
       pt.cex = 2, # Tamaño de los cuadrados
       cex = 1.32) # Cambiar el tamaño del texto

# Añadir valores sobre las barras
text_labels <- columna2

# Ajustar posición de las etiquetas
text(x = bpc, 
     y = text_labels + 2.5, 
     labels = format(text_labels),
     cex = 1.2, 
     pos = 3) # pos = 3 coloca el texto arriba de las barras

In [ ]:
mosaicplot(aux,
           main="Gráfico de Mosaico",
           color =c("darkblue","green"),
           border = "black",
           xlab = "Desepeño", # label for x-axis
          ylab = "Turno", # label for y-axis
           cex.axis=par("cex"))

Tablas contingencia de Lengua por turnos en cuarto grado.

In [ ]:
primero_lengua <- filter(Uruguay_Primaria, Año_cursado== 4, asignatura== "LENGUA")
primero_lengua <- subset(primero_lengua, Turno != "ROTATIVO" & Turno != "COMPLETO") #sacamos los turnos rotativo y completo
                                                                                    #porque no cumple con tener minimo 5 "muestras" en cada celda
tabla_contingencia_primero <- table(primero_lengua$Turno, primero_lengua$Desempenio)

tabla_contingencia_primero

In [ ]:
#Tuve que cambiar un poco las tablas por que antes no daban bien los numeros de Total
#porcentaje total
cat("\n","Porcentaje Total","\n")
porcentaje_total <- round(prop.table(tabla_contingencia_primero) * 100,1)
porcentaje_total<-cbind(porcentaje_total,"Total"=margin.table(porcentaje_total, 1))
porcentaje_total<-rbind(porcentaje_total,"Total"=margin.table(porcentaje_total, 2))
print(porcentaje_total)

#porcentaje por fila
cat("\n","Porcentaje Fila","\n")
porcentaje_fila<-round(prop.table(rbind(tabla_contingencia_primero,"Total"=margin.table(tabla_contingencia_primero, 2)),1)*100,1)
porcentaje_fila<-cbind(porcentaje_fila,"Total"=margin.table(porcentaje_fila, 1))
print(porcentaje_fila)

#porcentaje por columna
cat("\n","Porcentaje Columna", "\n")
porcentaje_columna<-round(prop.table(cbind(tabla_contingencia_primero,"Total"=margin.table(tabla_contingencia_primero, 1)),2)*100,1)
porcentaje_columna<-rbind(porcentaje_columna,"Total"=margin.table(porcentaje_columna, 2))
print(porcentaje_columna)

In [ ]:
porcentaje_fila <- porcentaje_fila[rownames(porcentaje_fila) != "Total", ]
porcentaje_fila <- porcentaje_fila[, colnames(porcentaje_fila) != "Total"]

porcentaje_columna <- porcentaje_columna[rownames(porcentaje_columna) != "Total", ]
porcentaje_columna <- porcentaje_columna[, colnames(porcentaje_columna) != "Total"]

In [ ]:
#test de chi-cuadrado
cat("\n","TEST de chi cuadrado")
resultado_chi <- chisq.test(tabla_contingencia_primero)
resultado_chi

#test de asosiacion cramerV
cat("\n","test de cramerV")
CramerV(tabla_contingencia_primero)

#tabla de valores esperados
cat("\n","Valores esperados")
round(resultado_chi$expected,0)

#residuos
cat("\n","Residuos")
round(resultado_chi$residuals, 1)

In [ ]:
aux <- t(porcentaje_fila)
bpc<-barplot(aux,
        main = "Gráfico de Contigencia por Fila",
        beside = TRUE, # Barras agrupadas
        space = c(0.1, 2.5), # Espacio primer elemento es el espacio entre barras de cada grupo (0.1) y el segundo el espacio entre grupos (2.5).
        ylim = c(0,100),
        cex.axis = 1.2, #tamaño etiquetas eje y
        cex.names=1.5, #tamaño etiqueta eje x
        cex.main=1.5, #tamaño título gráfico
        col=c("darkblue","green", "yellow","#364972","#6E1772","#1E7230"),
        xlab = "Turno",
        cex.lab=1.5, #tamaño  xlab ylab
        ylab = "Porcentaje",
       col.lab = "blue") # Color xlab ylab

#leyenda
legend("topright", inset = c(0, 0), # Ajusta la posición de la leyenda
       legend = rownames(aux),
       title = "Desempeño", # Título
       title.adj = 0.5,          # Ajuste horizontal del título
       title.col = "black",       # Color del título
       pch = 15,  # Cambia a cuadrados
       col = c("darkblue", "green", "yellow", "#364972", "#6E1772", "#1E7230"),
       pt.cex = 2,  # Tamaño de los cuadrados
       cex = 1.32)  # Cambiar el tamaño del texto

# valores sobre barras
TC <- aux
text(bpc, TC+2.5,format(TC), cex=1.2)


In [ ]:
df <- as.data.frame(as.table(porcentaje_columna))
colnames(df) <- c("Var1", "Var2", "Freq")

ggplot(data=df,aes(x=Var2,y=Freq,fill=Var1))+
  geom_bar(stat="identity",colour="blue")+
    scale_fill_manual(name= "Turnos", values=c("#5F9EA0","#1C7202"))+
   geom_text(aes(label=Freq),position=position_stack(vjust=0.41),size = 4, color = "red")+
  xlab("Desempeño")+
  ylab("Porcentaje")+
 ggtitle("Gráfico de Contigencia por columna") +
 theme(axis.title = element_text(size = 15, color = "black", face = "bold"))+
 theme(title = element_text(size = 15, color = "black", face = "bold"))+
  theme(plot.title = element_text(hjust = 0.5)) +#centra titulo
 theme(legend.text = element_text(size = 12, color = "black", face= "bold"))+
 theme(axis.text = element_text(size = 10, color = "black", face= "bold"))


Tablas contingencia de Matematica por turnos en primer grado.

In [ ]:
primero_mate <- filter(Uruguay_Primaria, Año_cursado== 1, asignatura== "MATEMATICA")
primero_mate <- subset(primero_mate, Turno != "ROTATIVO" & Turno != "COMPLETO") #sacamos los turnos rotativo y completo
                                                                                    #porque no cumple con tener minimo 5 "muestras" en cada celda
tabla_contigencia_1_mate <- table(primero_mate$Turno, primero_mate$Desempenio)
tabla_contigencia_1_mate

In [ ]:
#porcentaje total
cat("\n","Porcentaje Total","\n")
porcentaje_total <- round(prop.table(tabla_contigencia_1_mate) * 100,1)
porcentaje_total<-cbind(porcentaje_total,"Total"=margin.table(porcentaje_total, 1))
porcentaje_total<-rbind(porcentaje_total,"Total"=margin.table(porcentaje_total, 2))
print(porcentaje_total)

#porcentaje por fila
cat("\n","Porcentaje Fila","\n")
porcentaje_fila<-round(prop.table(rbind(tabla_contigencia_1_mate,"Total"=margin.table(tabla_contigencia_1_mate, 2)),1)*100,1)
porcentaje_fila<-cbind(porcentaje_fila,"Total"=margin.table(porcentaje_fila, 1))
print(porcentaje_fila)

#porcentaje por columna
cat("\n","Porcentaje Columna", "\n")
porcentaje_columna<-round(prop.table(cbind(tabla_contigencia_1_mate,"Total"=margin.table(tabla_contigencia_1_mate, 1)),2)*100,1)
porcentaje_columna<-rbind(porcentaje_columna,"Total"=margin.table(porcentaje_columna, 2))
print(porcentaje_columna)


In [ ]:
porcentaje_fila <- porcentaje_fila[rownames(porcentaje_fila) != "Total", ]
porcentaje_fila <- porcentaje_fila[, colnames(porcentaje_fila) != "Total"]

porcentaje_columna <- porcentaje_columna[rownames(porcentaje_columna) != "Total", ]
porcentaje_columna <- porcentaje_columna[, colnames(porcentaje_columna) != "Total"]

In [ ]:
#test de chi-cuadrado
cat("\n","TEST de chi cuadrado")
resultado_chi <- chisq.test(tabla_contigencia_1_mate)
resultado_chi

#test de asosiacion cramerV
cat("\n","test de cramerV")
CramerV(tabla_contigencia_1_mate)

#tabla de valores esperados
cat("\n","Valores esperados")
round(resultado_chi$expected,0)

#residuos
cat("\n","Residuos")
round(resultado_chi$residuals, 1)

In [ ]:
df <- as.data.frame(as.table(porcentaje_columna))
colnames(df) <- c("Var1", "Var2", "Freq")

ggplot(data=df,aes(x=Var2,y=Freq,fill=Var1))+
  geom_bar(stat="identity",colour="blue")+
    scale_fill_manual(name= "Turnos", values=c("#5F9EA0","#1C7202"))+
   geom_text(aes(label=Freq),position=position_stack(vjust=0.41),size = 4, color = "red")+
  xlab("Desempeño")+
  ylab("Porcentaje")+
 ggtitle("Gráfico de Contigencia por columna") +
 theme(axis.title = element_text(size = 15, color = "black", face = "bold"))+
 theme(title = element_text(size = 15, color = "black", face = "bold"))+
  theme(plot.title = element_text(hjust = 0.5)) +#centra titulo
 theme(legend.text = element_text(size = 12, color = "black", face= "bold"))+
 theme(axis.text = element_text(size = 10, color = "black", face= "bold"))


In [ ]:
aux <- t(porcentaje_fila)
bpc<-barplot(aux,
        main = "Gráfico de Contigencia por Fila",
        beside = TRUE, # Barras agrupadas
        space = c(0.1, 2.5), # Espacio primer elemento es el espacio entre barras de cada grupo (0.1) y el segundo el espacio entre grupos (2.5).
        ylim = c(0,100),
        cex.axis = 1.2, #tamaño etiquetas eje y
        cex.names=1.5, #tamaño etiqueta eje x
        cex.main=1.5, #tamaño título gráfico
        col=c("darkblue","green", "yellow","#364972","#6E1772","#1E7230"),
        xlab = "Turno",
        cex.lab=1.5, #tamaño  xlab ylab
        ylab = "Porcentaje",
       col.lab = "blue") # Color xlab ylab

#leyenda
legend("topright", inset = c(0, 0), # Ajusta la posición de la leyenda
       legend = rownames(aux),
       title = "Desempeño", # Título
       title.adj = 0.5,          # Ajuste horizontal del título
       title.col = "black",       # Color del título
       pch = 15,  # Cambia a cuadrados
       col = c("darkblue", "green", "yellow", "#364972", "#6E1772", "#1E7230"),
       pt.cex = 2,  # Tamaño de los cuadrados
       cex = 1.32)  # Cambiar el tamaño del texto

# valores sobre barras
TC <- aux
text(bpc, TC+2.5,format(TC), cex=1.2)


Tablas contingencia de Matematica por turnos en cuarto grado.

In [ ]:
primero_mate <- filter(Uruguay_Primaria, Año_cursado== 4, asignatura== "MATEMATICA")
primero_mate <- subset(primero_mate, Turno != "ROTATIVO" & Turno != "COMPLETO") #sacamos los turnos rotativo y completo
                                                                                    #porque no cumple con tener minimo 5 "muestras" en cada celda
tabla_contigencia_1_mate <- table(primero_mate$Turno, primero_mate$Desempenio)
tabla_contigencia_1_mate

In [ ]:
#porcentaje total
cat("\n","Porcentaje Total","\n")
porcentaje_total <- round(prop.table(tabla_contigencia_1_mate) * 100,1)
porcentaje_total<-cbind(porcentaje_total,"Total"=margin.table(porcentaje_total, 1))
porcentaje_total<-rbind(porcentaje_total,"Total"=margin.table(porcentaje_total, 2))
print(porcentaje_total)

#porcentaje por fila
cat("\n","Porcentaje Fila","\n")
porcentaje_fila<-round(prop.table(rbind(tabla_contigencia_1_mate,"Total"=margin.table(tabla_contigencia_1_mate, 2)),1)*100,1)
porcentaje_fila<-cbind(porcentaje_fila,"Total"=margin.table(porcentaje_fila, 1))
print(porcentaje_fila)

#porcentaje por columna
cat("\n","Porcentaje Columna", "\n")
porcentaje_columna<-round(prop.table(cbind(tabla_contigencia_1_mate,"Total"=margin.table(tabla_contigencia_1_mate, 1)),2)*100,1)
porcentaje_columna<-rbind(porcentaje_columna,"Total"=margin.table(porcentaje_columna, 2))
print(porcentaje_columna)


In [ ]:
porcentaje_fila <- porcentaje_fila[rownames(porcentaje_fila) != "Total", ]
porcentaje_fila <- porcentaje_fila[, colnames(porcentaje_fila) != "Total"]

porcentaje_columna <- porcentaje_columna[rownames(porcentaje_columna) != "Total", ]
porcentaje_columna <- porcentaje_columna[, colnames(porcentaje_columna) != "Total"]

In [ ]:
#test de chi-cuadrado
cat("\n","TEST de chi cuadrado")
resultado_chi <- chisq.test(tabla_contigencia_1_mate)
resultado_chi

#test de asosiacion cramerV
cat("\n","test de cramerV")
CramerV(tabla_contigencia_1_mate)

#tabla de valores esperados
cat("\n","Valores esperados")
round(resultado_chi$expected,0)

#residuos
cat("\n","Residuos")
round(resultado_chi$residuals, 1)

In [ ]:
df_turnos <- as.data.frame(as.table(porcentaje_columna))
colnames(df_turnos) <- c("Var1", "Var2", "Freq")

ggplot(data=df_turnos,aes(x=Var2,y=Freq,fill=Var1))+
  geom_bar(stat="identity",colour="blue")+
    scale_fill_manual(name= "Turnos", values=c("#5F9EA0","#1C7202"))+
   geom_text(aes(label=Freq),position=position_stack(vjust=0.41),size = 4, color = "red")+
  xlab("Desempeño")+
  ylab("Porcentaje")+
 ggtitle("Gráfico de Contigencia por columna") +
 theme(axis.title = element_text(size = 15, color = "black", face = "bold"))+
 theme(title = element_text(size = 15, color = "black", face = "bold"))+
  theme(plot.title = element_text(hjust = 0.5)) +#centra titulo
 theme(legend.text = element_text(size = 12, color = "black", face= "bold"))+
 theme(axis.text = element_text(size = 10, color = "black", face= "bold"))


In [ ]:
aux <- t(porcentaje_fila)
bpc<-barplot(aux,
        main = "Gráfico de Contigencia por Fila",
        beside = TRUE, # Barras agrupadas
        space = c(0.1, 2.5), # Espacio primer elemento es el espacio entre barras de cada grupo (0.1) y el segundo el espacio entre grupos (2.5).
        ylim = c(0,100),
        cex.axis = 1.2, #tamaño etiquetas eje y
        cex.names=1.5, #tamaño etiqueta eje x
        cex.main=1.5, #tamaño título gráfico
        col=c("darkblue","green", "yellow","#364972","#6E1772","#1E7230"),
        xlab = "Turno",
        cex.lab=1.5, #tamaño  xlab ylab
        ylab = "Porcentaje",
       col.lab = "blue") # Color xlab ylab

#leyenda
legend("topright", inset = c(0, 0), # Ajusta la posición de la leyenda
       legend = rownames(aux),
       title = "Desempeño", # Título
       title.adj = 0.5,          # Ajuste horizontal del título
       title.col = "black",       # Color del título
       pch = 15,  # Cambia a cuadrados
       col = c("darkblue", "green", "yellow", "#364972", "#6E1772", "#1E7230"),
       pt.cex = 2,  # Tamaño de los cuadrados
       cex = 1.32)  # Cambiar el tamaño del texto

# valores sobre barras
TC <- aux
text(bpc, TC+2.5,format(TC), cex=1.2)


tablas de contingencia por sector, Lengua de primer grado

In [ ]:
sector_lengua_1 <- filter(Uruguay_Primaria, Año_cursado== 1, asignatura== "LENGUA")

sector_lengua_1$EsPrivada <- ifelse(sector_lengua_1$EsPrivada == "N", "Publica","Privada")

tabla_contigencia_sect_len_1 <- table(sector_lengua_1$EsPrivada, sector_lengua_1$Desempenio)
tabla_contigencia_sect_len_1

In [ ]:
#porcentaje total
cat("\n","Porcentaje Total","\n")
porcentaje_total <- round(prop.table(tabla_contigencia_sect_len_1) * 100,1)
porcentaje_total<-cbind(porcentaje_total,"Total"=margin.table(porcentaje_total, 1))
porcentaje_total<-rbind(porcentaje_total,"Total"=margin.table(porcentaje_total, 2))
print(porcentaje_total)

#porcentaje por fila
cat("\n","Porcentaje Fila","\n")
porcentaje_fila<-round(prop.table(rbind(tabla_contigencia_sect_len_1,"Total"=margin.table(tabla_contigencia_sect_len_1, 2)),1)*100,1)
porcentaje_fila<-cbind(porcentaje_fila,"Total"=margin.table(porcentaje_fila, 1))
print(porcentaje_fila)

#porcentaje por columna
cat("\n","Porcentaje Columna", "\n")
porcentaje_columna<-round(prop.table(cbind(tabla_contigencia_sect_len_1,"Total"=margin.table(tabla_contigencia_sect_len_1, 1)),2)*100,1)
porcentaje_columna<-rbind(porcentaje_columna,"Total"=margin.table(porcentaje_columna, 2))
print(porcentaje_columna)


In [ ]:
porcentaje_fila <- porcentaje_fila[rownames(porcentaje_fila) != "Total", ]
porcentaje_fila <- porcentaje_fila[, colnames(porcentaje_fila) != "Total"]

porcentaje_columna <- porcentaje_columna[rownames(porcentaje_columna) != "Total", ]
porcentaje_columna <- porcentaje_columna[, colnames(porcentaje_columna) != "Total"]

In [ ]:
#test de chi-cuadrado
cat("\n","TEST de chi cuadrado")
resultado_chi <- chisq.test(tabla_contigencia_sect_len_1)
resultado_chi

#test de asosiacion cramerV
cat("\n","test de cramerV")
CramerV(tabla_contigencia_sect_len_1)

#tabla de valores esperados
cat("\n","Valores esperados")
round(resultado_chi$expected,0)

#residuos
cat("\n","Residuos")
round(resultado_chi$residuals, 1)

In [ ]:
#               Privada Publica
#  insuficiente       3      33
#  regular            6      76
#  aprobado          18     118
#  bueno             43     113
#  muy bueno         71     149
#  distinguido       92     143
prop.test(x=c(3,6,18,43,71,92), n=c(33,76,118,113,149,143), alternative="two.side", conf.level=0.95, correct=FALSE)

In [ ]:
#h0: No hay diferencia en la proporción de alumnos con calificaciones "muy bueno" y "distinguido" entre las escuelas privadas y públicas.
#h1: Hay una diferencia en la proporción de alumnos con calificaciones "muy bueno" y "distinguido" entre las escuelas privadas y públicas.

#sumamos las mejores categorias por tipo de gestion
privada_mejores_notas <- sum(c(71, 92))
publica_mejores_notas <- sum(c(149, 143))

#suma de todas las categorias por tipo de gestion
cat("Esc privada:",3 + 6 + 18 + 43 + 71 + 92,"\n")
cat("Esc publica:",33 + 76 + 118 + 113 + 149 + 143)

# Realizar la prueba para la categoría "muy bueno" y "distinguido"
prop.test(x = c(privada_mejores_notas, publica_mejores_notas), n = c(233, 632), alternative = "two.sided", 
                    conf.level = 0.95, 
                    correct = FALSE)
#se rechaza h0. p-value = 5.395e-10
#el intervalo de confianza va desde 0.167 hasta 0.308, lo que significa que podemos estar 95% seguros cae entre ese % y no incluye el 0, por lo que la diferencia no puede ser 0%.

In [ ]:
df_sector <- as.data.frame(as.table(porcentaje_columna))
colnames(df_sector) <- c("Var1", "Var2", "Freq")

ggplot(data=df_sector,aes(x=Var2,y=Freq,fill=Var1))+
  geom_bar(stat="identity",colour="blue")+
    scale_fill_manual(name= "Gestion", values=c("#5F9EA0","#1C7202"))+
   geom_text(aes(label=Freq),position=position_stack(vjust=0.41),size = 4, color = "red")+
  xlab("Desempeño")+
  ylab("Porcentaje")+
 ggtitle("Gráfico de Contigencia por columna") +
 theme(axis.title = element_text(size = 15, color = "black", face = "bold"))+
 theme(title = element_text(size = 15, color = "black", face = "bold"))+
  theme(plot.title = element_text(hjust = 0.5)) +#centra titulo
 theme(legend.text = element_text(size = 12, color = "black", face= "bold"))+
 theme(axis.text = element_text(size = 10, color = "black", face= "bold"))


In [ ]:
aux <- t(porcentaje_fila)
bpc<-barplot(aux,
        main = "Gráfico de Contigencia por Fila",
        beside = TRUE, # Barras agrupadas
        space = c(0.1, 2.5), # Espacio primer elemento es el espacio entre barras de cada grupo (0.1) y el segundo el espacio entre grupos (2.5).
        ylim = c(0,100),
        cex.axis = 1.2, #tamaño etiquetas eje y
        cex.names=1.5, #tamaño etiqueta eje x
        cex.main=1.5, #tamaño título gráfico
        col=c("darkblue","green", "yellow","#364972","#6E1772","#1E7230"),
        xlab = "Turno",
        cex.lab=1.5, #tamaño  xlab ylab
        ylab = "Porcentaje",
       col.lab = "blue") # Color xlab ylab

#leyenda
legend("topright", inset = c(0, 0), # Ajusta la posición de la leyenda
       legend = rownames(aux),
       title = "Desempeño", # Título
       title.adj = 0.5,          # Ajuste horizontal del título
       title.col = "black",       # Color del título
       pch = 15,  # Cambia a cuadrados
       col = c("darkblue", "green", "yellow", "#364972", "#6E1772", "#1E7230"),
       pt.cex = 2,  # Tamaño de los cuadrados
       cex = 1.32)  # Cambiar el tamaño del texto

# valores sobre barras
TC <- aux
text(bpc, TC+2.5,format(TC), cex=1.2)


tablas de contingencia por sector, Lengua de cuarto grado

In [ ]:
sector_lengua_1 <- filter(Uruguay_Primaria, Año_cursado== 4, asignatura== "LENGUA")

sector_lengua_1$EsPrivada <- ifelse(sector_lengua_1$EsPrivada == "N", "Publica","Privada")

tabla_contigencia_sect_len_1 <- table(sector_lengua_1$EsPrivada, sector_lengua_1$Desempenio)
tabla_contigencia_sect_len_1

In [ ]:
#porcentaje total
cat("\n","Porcentaje Total","\n")
porcentaje_total <- round(prop.table(tabla_contigencia_sect_len_1) * 100,1)
porcentaje_total<-cbind(porcentaje_total,"Total"=margin.table(porcentaje_total, 1))
porcentaje_total<-rbind(porcentaje_total,"Total"=margin.table(porcentaje_total, 2))
print(porcentaje_total)

#porcentaje por fila
cat("\n","Porcentaje Fila","\n")
porcentaje_fila<-round(prop.table(rbind(tabla_contigencia_sect_len_1,"Total"=margin.table(tabla_contigencia_sect_len_1, 2)),1)*100,1)
porcentaje_fila<-cbind(porcentaje_fila,"Total"=margin.table(porcentaje_fila, 1))
print(porcentaje_fila)

#porcentaje por columna
cat("\n","Porcentaje Columna", "\n")
porcentaje_columna<-round(prop.table(cbind(tabla_contigencia_sect_len_1,"Total"=margin.table(tabla_contigencia_sect_len_1, 1)),2)*100,1)
porcentaje_columna<-rbind(porcentaje_columna,"Total"=margin.table(porcentaje_columna, 2))
print(porcentaje_columna)


In [ ]:
porcentaje_fila <- porcentaje_fila[rownames(porcentaje_fila) != "Total", ]
porcentaje_fila <- porcentaje_fila[, colnames(porcentaje_fila) != "Total"]

porcentaje_columna <- porcentaje_columna[rownames(porcentaje_columna) != "Total", ]
porcentaje_columna <- porcentaje_columna[, colnames(porcentaje_columna) != "Total"]

In [ ]:
#test de chi-cuadrado
cat("\n","TEST de chi cuadrado")
resultado_chi <- chisq.test(tabla_contigencia_sect_len_1)
resultado_chi

#test de asosiacion cramerV
cat("\n","test de cramerV")
CramerV(tabla_contigencia_sect_len_1)

#tabla de valores esperados
cat("\n","Valores esperados")
round(resultado_chi$expected,0)

#residuos
cat("\n","Residuos")
round(resultado_chi$residuals, 1)

In [ ]:
#h0: No hay diferencia en la proporción de alumnos con calificaciones "muy bueno" y "distinguido" entre las escuelas privadas y públicas.
#h1: Hay una diferencia en la proporción de alumnos con calificaciones "muy bueno" y "distinguido" entre las escuelas privadas y públicas.

#sumamos las mejores categorias por tipo de gestion
privada_mejores_notas <- sum(c(105, 82))
publica_mejores_notas <- sum(c(153, 119))

#suma de todas las categorias por tipo de gestion
cat("Esc privada:",2 + 11 + 19 + 54 + 105 + 82,"\n")
cat("Esc publica:",17 + 90 + 125 + 172 + 153 + 119)

# Realizar la prueba para la categoría "muy bueno" y "distinguido"
prop.test(x = c(privada_mejores_notas, publica_mejores_notas), n = c(273, 676), alternative = "two.sided", 
                    conf.level = 0.95, 
                    correct = FALSE)


In [ ]:
df_sector <- as.data.frame(as.table(porcentaje_columna))
colnames(df_sector) <- c("Var2", "Var1", "Freq")

ggplot(data=df_sector,aes(x=Var1,y=Freq,fill=Var2))+
  geom_bar(stat="identity",colour="blue")+
    scale_fill_manual(name= "Gestion", values=c("#5F9EA0","#1C7202"))+
   geom_text(aes(label=Freq),position=position_stack(vjust=0.41),size = 4, color = "red")+
  xlab("Desempeño")+
  ylab("Porcentaje")+
 ggtitle("Gráfico de Contigencia por columna") +
 theme(axis.title = element_text(size = 15, color = "black", face = "bold"))+
 theme(title = element_text(size = 15, color = "black", face = "bold"))+
  theme(plot.title = element_text(hjust = 0.5)) +#centra titulo
 theme(legend.text = element_text(size = 12, color = "black", face= "bold"))+
 theme(axis.text = element_text(size = 10, color = "black", face= "bold"))


In [ ]:
aux <- t(porcentaje_fila)
bpc<-barplot(aux,
        main = "Gráfico de Contigencia por Fila",
        beside = TRUE, # Barras agrupadas
        space = c(0.1, 2.5), # Espacio primer elemento es el espacio entre barras de cada grupo (0.1) y el segundo el espacio entre grupos (2.5).
        ylim = c(0,100),
        cex.axis = 1.2, #tamaño etiquetas eje y
        cex.names=1.5, #tamaño etiqueta eje x
        cex.main=1.5, #tamaño título gráfico
        col=c("darkblue","green", "yellow","#364972","#6E1772","#1E7230"),
        xlab = "Turno",
        cex.lab=1.5, #tamaño  xlab ylab
        ylab = "Porcentaje",
       col.lab = "blue") # Color xlab ylab

#leyenda
legend("topright", inset = c(0, 0), # Ajusta la posición de la leyenda
       legend = rownames(aux),
       title = "Desempeño", # Título
       title.adj = 0.5,          # Ajuste horizontal del título
       title.col = "black",       # Color del título
       pch = 15,  # Cambia a cuadrados
       col = c("darkblue", "green", "yellow", "#364972", "#6E1772", "#1E7230"),
       pt.cex = 2,  # Tamaño de los cuadrados
       cex = 1.32)  # Cambiar el tamaño del texto

# valores sobre barras
TC <- aux
text(bpc, TC+2.5,format(TC), cex=1.2)


tablas de contingencia de Matematica y Lengua (cuarto grado)

In [ ]:
primer_ano <- filter(Uruguay_Primaria, (asignatura == "LENGUA" | asignatura== "MATEMATICA") & Año_cursado== 4)

documentos_completos <- primer_ano %>%
  group_by(Documento) %>%
  filter(n_distinct(asignatura) == 2) %>%
  distinct(Documento)

#filtramos para tener los estudiantes que hacen las dos materias
primer_anio_cm <- Uruguay_Primaria %>% filter(Documento %in% documentos_completos$Documento)

aux <- filter(primer_anio_cm, (asignatura == "LENGUA" | asignatura== "MATEMATICA") & Año_cursado== 4)

tabla_con_mate_lengua <- table(aux$asignatura, aux$Desempenio)
tabla_con_mate_lengua


In [ ]:
#porcentaje total
cat("\n","Porcentaje Total","\n")
porcentaje_total <- round(prop.table(tabla_con_mate_lengua) * 100,1)
porcentaje_total<-cbind(porcentaje_total,"Total"=margin.table(porcentaje_total, 1))
porcentaje_total<-rbind(porcentaje_total,"Total"=margin.table(porcentaje_total, 2))
print(porcentaje_total)

#porcentaje por fila
cat("\n","Porcentaje Fila","\n")
porcentaje_fila<-round(prop.table(rbind(tabla_con_mate_lengua,"Total"=margin.table(tabla_con_mate_lengua, 2)),1)*100,1)
porcentaje_fila<-cbind(porcentaje_fila,"Total"=margin.table(porcentaje_fila, 1))
print(porcentaje_fila)

#porcentaje por columna
cat("\n","Porcentaje Columna", "\n")
porcentaje_columna<-round(prop.table(cbind(tabla_con_mate_lengua,"Total"=margin.table(tabla_con_mate_lengua, 1)),2)*100,1)
porcentaje_columna<-rbind(porcentaje_columna,"Total"=margin.table(porcentaje_columna, 2))
print(porcentaje_columna)


In [ ]:
porcentaje_fila <- porcentaje_fila[rownames(porcentaje_fila) != "Total", ]
porcentaje_fila <- porcentaje_fila[, colnames(porcentaje_fila) != "Total"]

porcentaje_columna <- porcentaje_columna[rownames(porcentaje_columna) != "Total", ]
porcentaje_columna <- porcentaje_columna[, colnames(porcentaje_columna) != "Total"]

In [ ]:
#test de chi-cuadrado
cat("\n","TEST de chi cuadrado")
resultado_chi <- chisq.test(tabla_con_mate_lengua)
resultado_chi

#test de asosiacion cramerV
cat("\n","test de cramerV")
CramerV(tabla_con_mate_lengua)

#tabla de valores esperados
cat("\n","Valores esperados")
round(resultado_chi$expected,0)

#residuos
cat("\n","Residuos")
round(resultado_chi$residuals, 1)

In [ ]:
#sumando los insufcientes, regular, etc.
cat("lengua total:",17 + 100 + 138 + 215 + 249 + 196,"\n") #lengua
cat("matematica total:",11 + 90 + 123 + 225 + 250 + 216) #mate

In [ ]:
#h0: Las proporciones de estudiantes que lograron un "aprobado" (6 de promedio) en lengua y Matematica son iguales.
#h1: Las proporciones de estudiantes que lograron un "aprobado" en lengua y Matematica son diferentes.
prop.test(x=c(138,123), n=c(915,915), alternative="two.side", conf.level=0.95, correct=FALSE)

#No hay sufuciente informacion para rechazar H0, p-value 0.0782, ademas el intervalo de confianza (-0.0032 a 0.0609) incluye el 0

In [ ]:
df_materias <- as.data.frame(as.table(porcentaje_columna))
colnames(df_materias) <- c("Var2", "Var1", "Freq")

ggplot(data=df_materias,aes(x=Var1,y=Freq,fill=Var2))+
  geom_bar(stat="identity",colour="blue")+
    scale_fill_manual(name= "Materias", values=c("#5F9EA0","#1C7202"))+
   geom_text(aes(label=Freq),position=position_stack(vjust=0.41),size = 4, color = "red")+
  xlab("Desempeño")+
  ylab("Porcentaje")+
 ggtitle("Gráfico de Contigencia por columna") +
 theme(axis.title = element_text(size = 15, color = "black", face = "bold"))+
 theme(title = element_text(size = 15, color = "black", face = "bold"))+
  theme(plot.title = element_text(hjust = 0.5)) +#centra titulo
 theme(legend.text = element_text(size = 12, color = "black", face= "bold"))+
 theme(axis.text = element_text(size = 10, color = "black", face= "bold"))


In [ ]:
aux <- t(porcentaje_fila)
bpc<-barplot(aux,
        main = "Gráfico de Contigencia por Fila",
        beside = TRUE, # Barras agrupadas
        space = c(0.1, 2.5), # Espacio primer elemento es el espacio entre barras de cada grupo (0.1) y el segundo el espacio entre grupos (2.5).
        ylim = c(0,100),
        cex.axis = 1.2, #tamaño etiquetas eje y
        cex.names=1.5, #tamaño etiqueta eje x
        cex.main=1.5, #tamaño título gráfico
        col=c("darkblue","green", "yellow","#364972","#6E1772","#1E7230"),
        xlab = "Turno",
        cex.lab=1.5, #tamaño  xlab ylab
        ylab = "Porcentaje",
       col.lab = "blue") # Color xlab ylab

#leyenda
legend("topright", inset = c(0, 0), # Ajusta la posición de la leyenda
       legend = rownames(aux),
       title = "Desempeño", # Título
       title.adj = 0.5,          # Ajuste horizontal del título
       title.col = "black",       # Color del título
       pch = 15,  # Cambia a cuadrados
       col = c("darkblue", "green", "yellow", "#364972", "#6E1772", "#1E7230"),
       pt.cex = 2,  # Tamaño de los cuadrados
       cex = 1.32)  # Cambiar el tamaño del texto

# valores sobre barras
TC <- aux
text(bpc, TC+2.5,format(TC), cex=1.2)


In [ ]:
g2 <- ggplot(Uruguay_Primaria, aes(factor(Turno), Promedio))
g2 + geom_boxplot(aes(fill = factor(Turno)))

In [ ]:
datos_mate <- filter(Uruguay_Primaria_sin_epac, asignatura == "MATEMATICA") %>%
  mutate(EsPrivada = ifelse(EsPrivada == "N", "Pública", "Privada"))

colores <- c("#1f77b4", "#ff7f0e")

figurita <- plot_ly(datos_mate, x = ~Año_cursado, y = ~Promedio, color = ~EsPrivada, type = "box", colors = colores) %>% 
  layout(title = "Promedios de Matemática por grado y sector",
         boxmode = "group",
         xaxis = list(title = "Grado cursado"),
         yaxis = list(gridcolor = "grey"))

figurita

In [ ]:
library(GGally)
library(Hotelling)

In [ ]:
datos_lengua <- filter(Uruguay_Primaria_sin_epac, asignatura == "LENGUA") %>%
  mutate(EsPrivada = ifelse(EsPrivada == "N", "Pública", "Privada"))

colores <- c("#1f77b4", "#ff7f0e")

figurita <- plot_ly(datos_lengua, x = ~Año_cursado, y = ~Promedio, color = ~EsPrivada, type = "box", colors = colores) %>% 
  layout(title = "Promedios de Lengua por grado y sector",
         boxmode = "group",
         xaxis = list(title = "Grado cursado"),
         yaxis = list(gridcolor = "grey"))

figurita

In [ ]:
sexto_anio <- Uruguay_Primaria_sin_epac %>%
  filter(Año_cursado == 6 & asignatura %in% c("CIENCIAS SOCIALES", "MATEMATICA", "LENGUA", "CIENCIAS NATURALES"))

# Identificar los documentos de los alumnos que realizan las 4 asignaturas
documentos_completos <- sexto_anio %>%
  group_by(Documento) %>%
  filter(n_distinct(asignatura) == 4) %>%
  distinct(Documento)

# Filtrar el dataset original para obtener solo los registros de los alumnos que realizan las 4 asignaturas
sexto_anio_completo <- sexto_anio %>%
  filter(Año_cursado == 6 & Documento %in% documentos_completos$Documento & asignatura %in% c("CIENCIAS SOCIALES", "MATEMATICA", "LENGUA", "CIENCIAS NATURALES"))


promedios_alumnos_primera_parte <- sexto_anio_completo %>%
  group_by(Documento, asignatura,Turno) %>%
summarize(Promedio = mean(Promedio, na.rm = TRUE),.groups ="drop")  
head(promedios_alumnos)

promedios_alumnos <- promedios_alumnos_primera_parte %>%
    pivot_wider(names_from = asignatura, values_from = Promedio)
head(promedios_alumnos)

promedios_alumnos_sin_documento <- promedios_alumnos %>%
  select(-Documento)
promedios_alumnos_sin_documento 

# Crear dos datasets separados por turno
promedios_alumnos_manana <- promedios_alumnos_sin_documento %>%
  filter(Turno == "MAÑANA")
promedios_alumnos_manana

promedios_alumnos_tarde <- promedios_alumnos_sin_documento %>%
  filter(Turno == "TARDE") 

promedios_alumnos_tarde

In [ ]:
datoaux <- promedios_alumnos_manana[-1]
mardia2(datoaux)

In [ ]:
datos_alumnos_tarde<- promedios_alumnos_tarde[-1]
mardia2(datos_alumnos_tarde)

In [ ]:
head(promedios_alumnos_sin_documento)
tarde=filter(promedios_alumnos_sin_documento, Turno=="TARDE")[,2:5]
mañana=filter(promedios_alumnos_sin_documento, Turno=="MAÑANA")[,2:5]
prom.tarde =apply(tarde, 2, mean)
prom.tarde
prom.mañana= apply(mañana, 2, mean)
prom.mañana

s1 = cov(tarde)
s1
s2 = cov(mañana)
s2

n1 = nrow(tarde)
n1
n2 = nrow(mañana)
n2
p = ncol(tarde)

s = ((n1-1)*s1 + (n2-1)*s2) / (n1 + n2 - 2)
s

d2 = t(prom.tarde-prom.mañana)%*%solve(s)%*%(prom.tarde-prom.mañana)
d2
t2 = d2*((n1*n2) / (n1+n2))
t2
f = (n1+n2-p-1)/((n1+n2-2)*p) * t2
f
alfa = 0.05

qf(p=0.05, df1=5, df2=11, lower.tail=FALSE) #valor de constraste F

pf(f, p, n1+n2-p-1 ,lower.tail = F)

In [ ]:
#hoteling con libreria
resultado = hotelling.test(.~Turno, data = promedios_alumnos_sin_documento)
resultado 

In [ ]:
#MANOVA MULTIVARIANTE TURNOS
mod1<-manova(cbind(`CIENCIAS NATURALES`,`CIENCIAS SOCIALES`,LENGUA,MATEMATICA)~Turno,data=promedios_alumnos_sin_documento)
summary(mod1)

In [ ]:
summary.aov(mod1)

In [ ]:
promedios_alumnos_sin_documento <- promedios_alumnos_sin_documento %>% rename_with(make.names)

#promedios_alumnos_sin_documento

promedios_alumnos_sin_documento %>% group_by(Turno)

#no funciona como deberia, tendria que mostrarme cada materia por cada turno
library(rstatix) 
promedios_alumnos_sin_documento %>% shapiro_test(CIENCIAS.NATURALES,CIENCIAS.SOCIALES,LENGUA,MATEMATICA)

In [ ]:
#Supuestos de normalidad multivariante
mardia2(promedios_alumnos_sin_documento[, c("CIENCIAS.NATURALES", "CIENCIAS.SOCIALES", "LENGUA", "MATEMATICA")])

Realizamos gráfico de coordenadas paralelas para las materias troncales del turno mañana

In [ ]:
materias_demañana <- filter(Uruguay_Primaria_sin_epac, (asignatura == "MATEMATICA" | asignatura == "LENGUA"), Turno == "MAÑANA")
materias_demañana <- select(materias_demañana,NotasPrimerT, NotasSegundoT, NotasTercerT, asignatura)
head(materias_demañana,4)

In [ ]:
ggparcoord(data = materias_demañana,
           columns = 1:3,
           groupColumn = "asignatura",
           showPoints = TRUE,
           scale = "globalminmax")

Realizamos gráfico de coordenadas paralelas para las materias troncales del turno tarde

In [ ]:
materias_detarde <- filter(Uruguay_Primaria_sin_epac, (asignatura == "MATEMATICA" | asignatura == "LENGUA"), Turno == "TARDE")
materias_detarde <- select(materias_detarde, NotasPrimerT, NotasSegundoT, NotasTercerT, asignatura)

In [ ]:
ggparcoord(data = materias_detarde,
           columns = 1:3,
           groupColumn = "asignatura",
           showPoints = TRUE,
           scale = "globalminmax")

In [ ]:
library(scatterplot3d)
library(tidyverse)

In [ ]:
asignaturas_interes <- c("MATEMATICA", "LENGUA", "CIENCIAS NATURALES", "CIENCIAS SOCIALES")
PRIM_año_1_asignaturas <- filter(Uruguay_Primaria, Año_cursado == 1 & asignatura %in% asignaturas_interes)

medianas_df <- PRIM_año_1_asignaturas %>%
  group_by(Turno, asignatura) %>%
  summarize(mediana_promedio = median(Promedio, na.rm = TRUE)) %>%
  ungroup()

# Crear el gráfico de perfiles con ajustes de tema y etiquetas
ggplot(medianas_df, aes(x = asignatura, y = mediana_promedio, group = Turno, color = Turno)) +
  geom_line() +
  geom_point() +
  labs(title = "Gráfico de Perfiles de Promedios por Turno (Año Cursado 1)",
       x = "Asignatura",
       y = "Mediana de Promedios") +
  theme_minimal() +
  theme(
    axis.text.y = element_text(face = "bold"),
    axis.text.x = element_text(angle = 45, hjust = 1, face = "bold"),  # rotamos etiquetas del eje X
    plot.title = element_text(size = 14, face = "bold"), 
    plot.margin = margin(t = 10, r = 10, b = 10, l = 10, unit = "pt")  
  )

In [ ]:
# Filtrar los datos para el primer grado
primer_año<- Uruguay_Primaria_sin_epac %>%
  filter(Año_cursado == 1 & asignatura %in% c("MATEMATICA", "LENGUA", "CIENCIAS SOCIALES"))

# Identificar los documentos de los alumnos que realizan las 3 asignaturas
documentos_completos_primero <- primer_año %>%
  group_by(Documento) %>%
  filter(n_distinct(asignatura) == 3) %>%
  distinct(Documento)

# Filtrar el dataset original para obtener solo los registros de los alumnos que realizan las 3 asignaturas
primer_anio_completo <- primer_año %>%
  filter(Año_cursado == 1 & Documento %in% documentos_completos_primero$Documento & asignatura %in% c("MATEMATICA", "LENGUA", "CIENCIAS SOCIALES"))


In [ ]:
promedios_escuelas <- primer_anio_completo %>%
  group_by(CUE, asignatura, EsPrivada)


In [ ]:
promedios_escuelas <- promedios_escuelas %>%
  summarize(Promedio = mean(Promedio, na.rm = TRUE), .groups = 'drop') %>%
  pivot_wider(names_from = asignatura, values_from = Promedio)

In [ ]:
promedios_escuelas <- promedios_escuelas %>% rename_with(make.names)
promedios_escuelas %>% group_by(EsPrivada)

In [ ]:
  colors <- c("green", "red", "blue") 
  colors <- colors[as.factor(promedios_escuelas$EsPrivada)]
  with(promedios_escuelas, {
    s3d <- scatterplot3d(
      x = LENGUA,
      y = MATEMATICA,
      z = CIENCIAS.SOCIALES,
      color = colors,
      pch = 16,
      type = "h",
      main = "DIAGRAMA DE DISPERSION",
      xlab = "Prom Lengua",
      ylab = "Prom Matematica",
      zlab = "Prom Ciencias Naturales")

    # convert 3-D coords to 2D projection
    s3d.coords <- s3d$xyz.convert(LENGUA, MATEMATICA, CIENCIAS.SOCIALES)

    # plot text with 50% shrink and place to right of points
    text(s3d.coords$x,
        s3d.coords$y,
        labels = promedios_escuelas$CUE,
        cex = .9,
        pos = 4)

            # Agregar la leyenda
  legend("topright", 
         legend = c("No Privada", "Privada"), 
         col = c("green", "red"), 
         pch = 16,
         title = "Tipo de Escuela")
  
  })

In [ ]:
Cue_prom_leng_mat<- filter(Uruguay_Primaria_sin_epac, (asignatura =="LENGUA" | asignatura =="MATEMATICA"), Año_cursado == 1) %>%
          dplyr::select(CUE, Promedio)
aux <- Cue_prom_leng_mat %>% group_by(CUE) %>% summarise_all(median)
#medias_cue

caras<-faces(aux[2],face.type=1, main ="Escuelas", labels=aux$CUE,
    print.info = 1, nrow.plot = 5, ncol.plot = 5, plot.faces = TRUE, cex = 2)

In [ ]:
# Filtrar los datos para obtener solo los estudiantes únicos por Documento
datos_unicos <- Uruguay_Primaria_sin_epac %>% distinct(Documento, .keep_all = TRUE)

# Filtrar los datos para los años 1 a 6
datos_filtrados <- datos_unicos %>%
  filter(Año_cursado %in% 1:6)

# Crear el boxplot combinado usando facet_wrap para organizar por grado
ggplot(datos_filtrados, aes(x = EsPrivada, y = Promedio, fill = EsPrivada)) +
  geom_boxplot() +
  labs(title = "Comparación de Promedios en Escuelas Públicas y Privadas",
       x = "Tipo de Escuela",
       y = "Promedio") +
  scale_fill_manual(values = c("S" = "blue", "N" = "red"), labels = c("Publica", "Privada")) +
  theme_minimal() +
  theme(
    axis.text = element_text(size = 12),  # Tamaño de los números en los ejes
    strip.text = element_text(face = "bold")  # Títulos de los años en negrita
  ) +
  facet_wrap(~ Año_cursado, ncol = 2, scales = "free_y", labeller = label_both)

No tomar en cuenta.

In [ ]:
                                        #ignorar, solo ver como curiosidad, podes hasta voltear el grafico o hacerle zoom

# Calcular el vector de medias para cada escuela
medias_por_escuela <- Uruguay_Primaria_sin_epac %>%
  group_by(CUE, asignatura) %>%
  summarize(Media_Nota = mean(NotasPrimerT, na.rm = TRUE)) %>%
  spread(asignatura, Media_Nota)

fig <- plot_ly(medias_por_escuela, 
               x = ~MATEMATICA, 
               y = ~LENGUA, 
               z = ~INGLES, 
               color = ~CUE, 
               type = "scatter3d", 
               mode = "markers") %>%
  layout(title = "Promedios de Primer grado por Escuela",
         scene = list(xaxis = list(title = "Matematica"),
                      yaxis = list(title = "Lengua"),
                      zaxis = list(title = "ingles")))
fig
